In [13]:
from langchain_community.graphs import Neo4jGraph

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Access environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# Connection to Neo4j
graph = Neo4jGraph()

In [14]:
query = """
MATCH (f:File {sectionizer_type: "RELATED_TOPICS"}) - [] - (fs:FileSection) - [] - (fc:FileChunk)
RETURN DISTINCT f.extracted_text, f.id
LIMIT 25
"""

data = graph.query(query)


In [5]:
from dotenv import load_dotenv

load_dotenv(override=True)  

True

In [6]:
from langchain_aws import ChatBedrock

model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0),
    name="agent"
)

data 

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
import json

system_prompt = """
You are an expert in creating datasets with questions about municipal policy documents. Your task is to generate practical, outcome-oriented questions based on the content of the given texts. Remember that these texts will not be directly available in the final database. The questions should focus on concrete decisions, changes, or evaluations that are relevant to citizens.

For each block of 5 text entries (indexed from 0 to 24), create questions that capture the key points from ALL 5 entries. The output should be in the format of a Python dictionary where the keys are the indices of the text entries (0 to 24) and the values are lists of questions for each text.

IMPORTANT: Your response should ONLY contain the Python dictionary in valid JSON format, nothing else.

Example structure (not to be inherited):
{{
 "0": [
    "Welche konkreten Auswirkungen hat die Verschiebung der Ortschaftsratsitzung in Lehen im Juli 2024 für die Bürger?",
    "Wie können Bürger trotz der Terminänderung ihre Anliegen an den Ortsvorsteher von Lehen herantragen?"
 ],
 "1": [
    "Welche messbaren Ziele verfolgt die neue Kulturinitiative, die Ende September 2024 in Freiburg startet?",
    "Wie wirken sich die Schwerpunkte des Kulturamts Freiburg bei Eigenveranstaltungen auf das kulturelle Angebot für verschiedene Bevölkerungsgruppen aus?"
 ],
 ...
}}

Ensure that the questions are practice-oriented and refer to specific impacts, decisions, or changes that are relevant to citizens. The questions should be formulated in a way that they are understandable and relevant even without knowledge of the specific document.

ALL QUESTIONS MUST BE IN GERMAN.
"""

human_prompt = """
Chunk indices: {start_index} to {end_index}

Text content:
{text}

Please generate practical, outcome-oriented questions based on this content for ALL documents in this chunk. Focus on specific decisions, changes, or evaluations that are relevant to citizens. Return ONLY the Python dictionary in valid JSON format, with keys from {start_index} to {end_index}.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt),
])

all_questions = {}

for chunk_index in range(5):  # Process all 25 entries in 5 chunks of 5
    start_index = chunk_index * 5
    end_index = start_index + 4  # End index is inclusive
    chunk = data[start_index:end_index+1]
    text = "\n\n".join([f"Text {i}:\n{item['f.extracted_text']}" for i, item in enumerate(chunk, start=start_index)])
    formatted_prompt = prompt.format_messages(start_index=start_index, end_index=end_index, text=text)
    print(f"Processing chunk {chunk_index} (indices {start_index} to {end_index})")
    response = model.invoke(formatted_prompt)
    content = response.content
    try:
        questions = json.loads(content)
        all_questions.update(questions)
    except json.JSONDecodeError:
        print(f"Error decoding JSON for chunk {chunk_index}. Content: {content}")

# Ensure all indices from 0 to 24 are present
for i in range(25):
    if str(i) not in all_questions:
        all_questions[str(i)] = []

# Convert to the final format
final_questions = [{0: all_questions}]


Processing chunk 0 (indices 0 to 4)
Processing chunk 1 (indices 5 to 9)
Processing chunk 2 (indices 10 to 14)
Processing chunk 3 (indices 15 to 19)
Processing chunk 4 (indices 20 to 24)


In [75]:
all_questions

{'0': ['Welche konkreten Auswirkungen hat die Verschiebung der Ortschaftsratsitzung in Lehen vom 03.07.2024 auf den 25.07.2024 für die Bürger?',
  'Wie können Bürger ihre Anliegen zwischen den verschobenen Sitzungsterminen an den Ortsvorsteher herantragen?',
  'Welche Themen werden voraussichtlich in der Sitzung am 25.07.2024 behandelt?'],
 '1': ['Welche konkreten Maßnahmen plant das Kulturamt Freiburg, um die kulturelle Teilhabe verschiedener Bevölkerungsgruppen zu fördern?',
  "Wie wirken sich die neuen Förderinstrumente wie 'Experimentierraum – künstlerische Recherche' und 'Basisförderung' auf das kulturelle Angebot in Freiburg aus?",
  "Welche messbaren Ziele verfolgt das 'Kulturlabor Freiburg' ab September 2024 und wie können Bürger daran teilnehmen?"],
 '2': ['Wie haben sich die Übergangsquoten von Grundschulen zu weiterführenden Schulen in Weingarten im Vergleich zum Freiburger Durchschnitt entwickelt?',
  'Welche konkreten Maßnahmen plant die Stadt, um die Bildungschancen von S

In [81]:
# Function to map the index of questions to the corresponding index of the documents
def map_questions_to_documents(documents, questions):
    mapping = {}
    for idx, document in enumerate(documents):
        str_idx = str(idx)
        if str_idx in questions:
            mapping[idx] = {
                "questions": questions[str_idx],
                "document": document
            }
    return mapping

# Assume 'data' array is already available and provided elsewhere in the context
# Call the mapping function to map document indices to questions and documents
question_document_mapping = map_questions_to_documents(data, all_questions)

In [3]:
# question_document_mapping={0: {'questions': ['Welche konkreten Auswirkungen hat die Verschiebung der Ortschaftsratsitzung in Lehen vom 03.07.2024 auf den 25.07.2024 für die Bürger?',
#    'Wie können Bürger ihre Anliegen zwischen den verschobenen Sitzungsterminen an den Ortsvorsteher herantragen?',
#    'Welche Themen werden voraussichtlich in der Sitzung am 25.07.2024 behandelt?'],
#   'document': {'f.extracted_text': 'Stadt Freiburg im Breisgau · Ortsverwaltung Lehen\nBreisgauer Straße 61, D-79110 Freiburg\n\nOrtsverwaltung Lehen\n\nDezernat I\nAdresse: Breisgauer Straße 61\nD-79110 Freiburg i. Br.\nTelefon: 0761 / 88871 - 11\nTelefax: 0761 / 88871 - 20\nInternet: www.freiburg.de\nE-Mail*: ov-lehen@stadt.freiburg.de\n\nBekanntmachung\n\nIhr Zeichen/Schreiben vom        Unser Aktenzeichen        Ihnen schreibt\n                                 Frau Hess\n                                 Freiburg, den\n                                 14.06.2024\n\nLiebe Mitbürgerinnen und Mitbürger,\n\nfür den Juli waren zwei Termine für die Ortschaftsratsitzung des Ortschaftsrates Lehen angesetzt. Die Sitzung am Mittwoch, den 03.07.2024 entfällt und dafür findet die Sitzung am Donnerstag, 25.07.2024 statt.\n\nMit freundlichen Grüßen\n\ngez.\nBernhard Schätzle, Ortsvorsteher\n\nSprechzeiten: Montag bis Freitag 08.00 Uhr bis 11.30 Uhr Mittwoch von 17.00 Uhr bis 19.00 Uhr. Straßenbahn und Bus: Linien 19, 31, 32 Haltestelle Kirchbergstraße Sparkasse Freiburg - Nördlicher Breisgau: IBAN DE63 6805 0101 0002 0100 12 · BIC FRSPDE 66XXX\n\n*E-Mail-Adresse nur für formlose Mitteilungen ohne elektronische Signatur',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-OR_LE-112%7C1'}},
#  1: {'questions': ['Welche konkreten Maßnahmen plant das Kulturamt Freiburg, um die kulturelle Teilhabe verschiedener Bevölkerungsgruppen zu fördern?',
#    "Wie wirken sich die neuen Förderinstrumente wie 'Experimentierraum – künstlerische Recherche' und 'Basisförderung' auf das kulturelle Angebot in Freiburg aus?",
#    "Welche messbaren Ziele verfolgt das 'Kulturlabor Freiburg' ab September 2024 und wie können Bürger daran teilnehmen?"],
#   'document': {'f.extracted_text': 'Grundsätze und Perspektiven einer nachhaltig ausgerichteten Kunst- und Kulturförderung in Freiburg\n\nGrundsätze und Perspektiven einer nachhaltig ausgerichteten Kunst- und Kulturförderung in Freiburg\n\nNeustrukturierung der Website Kulturamt\nDokumentation des gesamten Prozesses\nAbruf aller Grundsätze\nDownload Grundsatzpapier\nInformationen zu aktuellen Veranstaltungen\nNewsletter\nIn Planung: Wissensbox\nKulturlabor Freiburg\n\nGrundsätze und Perspektiven einer nachhaltig ausgerichteten Kunst- und Kulturförderung in Freiburg\n\nWeiteres Vorgehen hinsichtlich Umsetzung\n\nFortführung Labor\n\nAusrichtung Eigenveranstaltungen des Kulturamtes\n\nAusrichtung und Weiterentwicklung der Förderinstrumente\n\nGrundsätze und Perspektiven einer nachhaltig ausgerichteten Kunst- und Kulturförderung in Freiburg\n\nLabor\nDenkfabrik Kunst und Transformation\nStart: Ende Sept. 2024\n\nWorkshops Umweltfreundliches Veranstalten und Klimaschutz im Gebäude\nFebruar, April und Sommer 2024\n\nNetzwerk- treffen\ndiskriminierungsfreie und inklusive Kulturarbeit\nseit 2024\n\nGrundsätze und Perspektiven einer nachhaltig ausgerichteten Kunst- und Kulturförderung in Freiburg\n\nFörderinstrumente\n\nNeue Ansätze\nKulturelle Teilhabe\nExperimentierraum – künstlerische Recherche\nBasisförderung\nFestivalförderung\n\nLeitlinien für inst. Förderung\nAnpassung Bewertung DUVA\nFördervoraussetzungen schaffen\nAufarbeitung lang bestehender Förderungen\n\nAusrichtung Eigenveranstaltungen\nSchwerpunkte: Teilhabe Diversität Vernetzung regionale Stärkung',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-KA-157%7C4'}},
#  2: {'questions': ['Wie haben sich die Übergangsquoten von Grundschulen zu weiterführenden Schulen in Weingarten im Vergleich zum Freiburger Durchschnitt entwickelt?',
#    'Welche konkreten Maßnahmen plant die Stadt, um die Bildungschancen von Schülern aus Weingarten zu verbessern?',
#    'Wie wirkt sich das geplante Bildungshaus ARS auf die Bildungssituation in Weingarten aus?'],
#   'document': {'f.extracted_text': 'Bildungsangebote in Weingarten\nAusschuss für Schule und Weiterbildung\nKinder- und Jugendhilfeausschuss\nMigrationsausschuss\n13.05.2024\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 1 12.06.2024\nDezernat II Amt für Schule und Bildung\nIM BREISGAU\n\n1. Ausgangspunkt\nDie Fraktionsanträge im Überblick\nDie Vorbereitung\n2. Die Datenlage\nDas 80%-Narrativ oder was wir tatsächlich zu wissen glauben\n6. Freiburger Bildungsbericht\n3. Erfahrungsberichte\nDer Blick auf Bildungschancen in Weingarten (Wodtke/ Hartwich)\nEine Außenklasse in Weingarten (Wodtke/ Baumgarten)\nBildungsberater_innen für Sinti_zze und Rom_nja (Schubert)\nElternbegleiter_innen (Rausch)\n4. Lösungsideen\nDas Bildungshaus AR (Schubert)\nÜbergangsbegleitung (Schubert/ Rausch/ Staenke)\nLernfördergruppen (Drehmann)\n\nAGENDA\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 2 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg IM BREISGAU\n\nDatenlage – Entwicklung der Übergangsquoten\nEntwicklung der Übergangsquoten an eine Realschule (in %)\n70  60  50  40  33,3  33,3  34,5  36,1  35,3  30  29,4  26,2  28,3  26,0  28,9  28,4  28,1  29,1  22,7  20  16,7  17,9  10  0  Albert-Schweitzer-Schule I  Lortzingschule  Pestalozzi-Grundschule  Adolf-Reichwein-Schule  2020  2021  2022  2023  23,0  21,3  19,8  18,2  Ø Freiburg\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 11 12.06.2024\nDezernat II Amt für Schule und Bildung\n° NH Freiburg IM BREISGAU\n\nDatenlage – Entwicklung der Übergangsquoten\nEntwicklung der Übergangsquoten an eine Gemeinschaftsschule (in %)\n70  60  50  40  35,2  30  29,4  30,3  20  16,7  12,8  13,4  14,6  19,6  10  10,0  5,2  4,0  3,7  8,9  0,0  0,0  1,0  0\nAlbert-Schweitzer-Schule I  Lortzingschule  Pestalozzi-Grundschule  Adolf-Reichwein-Schule  2020  2021  2022  2023  9,3  8,9  9,0  8,0  Ø Freiburg\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 12 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\nDatenlage – Entwicklung der Übergangsquoten\nEntwicklung der Übergangsquoten an eine Gesamtschule (in %)  70  60  50  40  30  28,9  31,1  20  18,8  10  8,3  0,0  0,0  0,0  2,6  0,0  1,6  0,0  3,7  2,6  2,5  0,0  2,5  0  Albert-Schweitzer-Schule I  Lortzingschule  Pestalozzi-Grundschule  Adolf-Reichwein-Schule  2020  2021  2022  2023  3,7  5,4  7,9  7,1  Ø Freiburg \nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 13 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg” IM BREISGAU\n\nDatenlage – Entwicklung der Übergangsquoten\n\nEntwicklung der Übergangsquoten an ein Gymnasium (in %)\n\n70  66,0  60  55,7  58,3  50  48,3  42,9  42,9  40  40,0  35,9  39,5  33,3  30  29,4  30,3  23,5  21,9  20  13,6  10  5,2  0  Albert-Schweitzer-Schule I  Lortzingschule  Pestalozzi-Grundschule  Adolf-Reichwein-Schule  2020  2021  2022  2023  54,6  55,5  54,0  56,4  Ø Freiburg\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 14 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg IM BREISGAU\n\nDie Datenlage – Schüler_innen aus Weingarten im SJ 2023/24 (N=801)\n\nAuf welche weiterführende Schulen gehen die Schüler_innen aus Weingarten? (N=801)\n\nStaudinger-Gesamtschule 19%  Sonstige 39%  Pestalozzi-Realschule 17%  Vigelius-Schule II - Gemeinschaftsschule 10%  Kepler-Gymnasium 15%\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 15 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg IM BREISGAU\n\nDie Datenlage – Schüler_innen aus Weingarten im SJ 2023/24 (N=801)\nWie viele der Schüler_innen an den Schulen kommen aus Weingarten?\n100%  90%  78 = 23,3%  152 = 13,4%  116 =  12,9%  80%  140 = 38,3%  70%  60%  50%  40%  257  983  781  30%  226  20%  10%  0%  Pestalozzi-Realschule  Vigelius-Schule II - Gemeinschaftsschule  Staudinger-Gesamtschule  Kepler-Gymnasium  SuS nicht aus Weingarten  SuS aus Weingarten\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 16 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\n6. Bildungsbericht\n\nDer 6. Bildungsbericht ist mit dem Themenschwerpunkt „Bildungsgerechtigkeit in Freiburg“ geplant. Geplant für 2026.\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 17 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg im BREISGAU\n\nDer Blick auf Bildungschancen in Weingarten (Frau Wodtke/ Herr Hartwich)\nEine Außenklasse in Weingarten (Frau Wodtke/ Herr Baumgarten)\nBildungsberater_innen für Sinti_zze und Rom_nja (Herr Schubert)\nElternbegleiter_innen (Frau Rausch)\n3. ERFAHRUNGSBERICHTE\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 18 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg IM BREISGAU\n\nDas Bildungshaus ARS (Herr Schubert)\nÜbergangsbegleitung (Frau Rausch, Herr Schubert, Herr Staenke)\nLernfördergruppen (Frau Drehmann/ Nachbarschaftswerk)\n4. LÖSUNGSANSÄTZE\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 19 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\nBildungsnetzwerk\n\nModellverbund „Durchgängige Sprachbildung“\nWeingartner AG für Soziales\nKinder- und Jugendnetz\nNetzwerk Zukunft-W\n…\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 20 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\n1. AUSGANGSPUNKT DIE FRAKTIONSANTRÄGE IM ÜBERBLICK\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 3 12.06.2024\nDezernat II Amt für Schule und Bildung\n\nDie Fraktionsanträge\n\nDie Daten betreffend\n• Übergang auf weiterf. Schulen\n• Schulabbrecher/ Schulwechlser\n• Schüler_innen aus Weingarten auf weiterf. Schulen\n• 6.Bildungsbericht\n\nDie Kinder und Jugendlichen betreffend\n• Übergangsbegleitung\n• Peer-to-peer-Begleitung\n• Fördermöglichkeiten im Quartier und in der Schule\n\nDie Strukturen betreffend\n• Außenklasse in Weingarten\n• Bildungsnetzwerk Weingarten\n• Fachtag\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 4 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\nDie Vorbereitung\nAdolf-Reichwein-Schule\nKepler Gymnasium\nStaudinger Gesamtschule\nVigeliusschule II\nPestalozzi Realschule\nNachbarschaftswerk\nDiakonie Südwest\nDeutsches Rote Kreuz\nKinder- und Jugendzentrum Weingarten\nForum Weingarten\nArbeiterwohlfahrt\nAmt für Schule und Bildung\nAmt für Kinder Jugend und Familie\nAmt für Soziales\nAmt für Migration und Integration\nDez III – Fachbereich Soziale Stadtentwicklung\nWegweiser Bildung\nFreiburger FC\nIN VIA Freiburg\nProf. Dr. Uta Meier-Gräwe (Weingartengespräche)\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 5 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\nDas 80%-Narrativ oder was wir wirklich zu wissen glauben\n\n2. DIE DATENLAGE\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 6 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg IM BREISGAU\n\nDas 80%-Narrativ\n\n„Aktuell schaffen es sehr viele Kinder mit einem Übergang in Realschule oder Gymnasium aus Weingarten nicht, sich auf dieser Schulart zu halten. In Gesprächen mit engagierten BürgerInnen wurden uns Zahlen von bis zu 80% genannt.“\n\n(Quelle: Aufsetzungsantrag für den ASW zum Thema „Übergangsbegleitung in Weingarten“; CDU, FL)\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 7 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg IM BREISGAU\n\nDas 80%-Narrativ\nKeine amtlichen, schulscharfen Statistiken in der gewünschten Feinheit vorhanden.\nAbfrage an Kepler-Gymnasium, Pestalozzi-Realschule, Staudinger-Gesamtschule und Vigeliusschule II Gemeinschaftsschule zu den Schüler_innen, die während des SJ 2022/23 die Schule vorzeitig verlassen haben. Die zitierten 80 Prozent sind für uns aufgrund der aktuellen Datenlage nicht nachvollziehbar. Die Daten aus der Abfrage zeigen ein sehr differenziertes Bild.\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 8 12.06.2024\nDezernat II Amt für Schule und Bildung\nim BREISGAU\n\nDatenlage – soziale Rahmenbedingungen\n\nStadtbezirk Einwohner_innen Anteil Personen mit (persönlichem) Migrationshintergrund Anteil Personen unter 18 Jahren Anteil Kinder in Bedarfsgemeinschaften unter 18 Jahre Anteil Fachhochschul- abschluss/Abitur\nBrühl-Beurbarung 2.666 43,1 20,1 25,7 60,4\nBrühl-Güterbahnhof 9.014 37,3 13,3 13,6 -\nBrühl-Industriegebiet 1.567 59,9 16,0 45,2 63,4\nHaslach-Egerten 8.179 35,1 16,4 19,2 72,5\nHaslach-Gartenstadt 8.240 36,3 18,5 16,1 57,7\nHaslach-Haid 3.949 31,7 16,9 21,4 52,9\nHaslach-Schildacker 2.003 66,7 26,8 20,8 57,7\nLandwasser 7.069 52,0 18,3 21,0 40,6\nWeingarten 11.584 54,5 21,0 34,2 54,2\nHerdern-Süd 6.473 23,8 16,9 5,9 92,1\nØ Freiburg 232.811 31,8 16,1 13,6 71,8\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 9 12.06.2024\nDezernat II Amt für Schule und Bildung\nFreiburg IM BREISGAU\n\nDatenlage – Entwicklung der Übergangsquoten\n\nEntwicklung der Übergangsquoten an eine Werkrealschule (in %)\n\n70\n60\n50\n40\n30\n20\n14,3\n16,7\n16,0\n13,6\n10\n5,9\n7,7\n6,9\n6,6\n5,0\n0,0\n3,7\n6,6\n0,0\n1,8\n3,1\n3,1\n0\nAlbert-Schweitzer-Schule I\nLortzingschule\nPestalozzi-Grundschule\nAdolf-Reichwein-Schule\n2020\n2021\n2022\n2023\n3,5\n3,6\n3,5\n3,7\nØ Freiburg\n\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 10 12.06.2024\n\nDezernat II Amt für Schule und Bildung\n\nFreiburg” IM BREISGAU',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-ASW-138%7C4'}},
#  3: {'questions': ['Welche Auswirkungen hat der Ausfall der Ortschaftsratsitzung am 05.06.2024 in Lehen auf laufende Projekte und Entscheidungen?',
#    'Wie können Bürger ihre Anliegen trotz des Ausfalls der Sitzung einbringen?',
#    'Wann findet die nächste reguläre Ortschaftsratsitzung in Lehen statt und welche Themen stehen auf der Tagesordnung?'],
#   'document': {'f.extracted_text': 'Stadt Freiburg im Breisgau · Ortsverwaltung Lehen\nBreisgauer Straße 61, D-79110 Freiburg\n\nBekanntmachung\n\nLiebe Mitbürgerinnen und Mitbürger,\n\ndie für den Juni terminierte Ortschaftsratsitzung des Ortschaftsrates Lehen am Mittwoch, den 05.06.2024, fällt aus.\n\nMit freundlichen Grüßen\n\nGez. Bernhard Schätzle\nOrtsvorsteher\n\nOrtsverwaltung Lehen\n\nDezernat I\nAdresse: Breisgauer Straße 61\nD-79110 Freiburg i. Br.\nTelefon: 0761 / 88871 - 11\nTelefax: 0761 / 88871 - 20\nInternet: www.freiburg.de\nE-Mail*: ov-lehen@stadt.freiburg.de\n\nIhr Zeichen/Schreiben vom\nUnser Aktenzeichen\nIhnen schreibt Frau Hess\nFreiburg, den 29.05.2024\n\nSprechzeiten: Montag bis Freitag 08.00 Uhr bis 11.30 Uhr Mittwoch von 17.00 Uhr bis 19.00 Uhr. Straßenbahn und Bus: Linien 19, 31, 32 Haltestelle Kirchbergstraße Sparkasse Freiburg - Nördlicher Breisgau: IBAN DE63 6805 0101 0002 0100 12 · BIC FRSPDE 66XXX\n\n*E-Mail-Adresse nur für formlose Mitteilungen ohne elektronische Signatur',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-OR_LE-111%7C1'}},
#  4: {'questions': ["Welche konkreten Änderungen beinhaltet die Fortentwicklung des Förderprogramms 'Klimafreundlich Wohnen' und wie profitieren Bürger davon?",
#    'Welche messbaren Ziele verfolgt die Stadt Freiburg mit dem European Energy Award (eea) und wie wirken sich diese auf den Alltag der Bürger aus?',
#    'Welche neuen Maßnahmen enthält das aktualisierte Energiepolitische Arbeitsprogramm und wie können Bürger dazu beitragen?'],
#   'document': {'f.extracted_text': 'Das Bürgermeisteramt der Stadt Freiburg im Breisgau - Dezernat I -\nFreiburg i. Br., 19.04.2024 Tel.: 0761/201-2017 E-Mail:Florian.Fletschinger@stadt.freiburg.de\nHerr Fletschinger\n\n3. Sitzung des Ausschusses für Umwelt und Klimaschutz\n\nMitglieder des Ausschusses für Umwelt und Klimaschutz\n\nIch lade zu der am\n\nMontag, 29. April 2024, 16:00 Uhr\n\nim Neuen Ratssaal des Rathauses stattfindenden Sitzung des Ausschusses für Umwelt und Klimaschutz ein.\n\nT a g e s o r d n u n g\n\nÖffentlicher Teil\n\n1. Förderprogramm "Klimafreundlich Wohnen": Fortentwicklung und Anpassung der Richtlinien\n- Drucksache G-24/059 - beratend\n(bereits zugestellt)\n\n2. European Energy Award (eea) - Internes Audit 2023, externes Audit 2024 und Aktualisierung des Energiepolitischen Arbeitsprogramms\n- Drucksache G-24/013 - beratend\n\n3. Bekanntgaben und Aktuelles\n\nH o r n Oberbürgermeister',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-UKA-28%7C1'}},
#  5: {'questions': ['Welche konkreten Änderungen im Spielplan des Theater Freiburg können Bürger für die Spielzeit 2024/2025 erwarten?',
#    'Wie wirkt sich die Neufassung der Satzung über die Führung des Theater Freiburg als Eigenbetrieb auf das kulturelle Angebot für die Bürger aus?',
#    'Welche Möglichkeiten haben Bürger, sich über die geplanten Änderungen im Theaterbetrieb zu informieren und ihre Meinung dazu zu äußern?'],
#   'document': {'f.extracted_text': 'Das Bürgermeisteramt der Stadt Freiburg im Breisgau - Dezernat I -\nFreiburg i. Br., 24.04.2024 Tel.: 0761/201-3013 E-Mail: Dez-iii@stadt.freiburg.de\nFrau Lebrecht\n\n2. Sitzung des Theaterausschusses\n\nMitglieder des Theaterausschusses\n\nIch lade zu der am\n\nDonnerstag, 2. Mai 2024, 16:00 Uhr\n\nim Neuen Ratssaal des Rathauses stattfindenden Sitzung des Theaterausschusses ein.\n\nT a g e s o r d n u n g\n\nÖffentlicher Teil\n\n1. Spielplan des Eigenbetrieb Theater Freiburg für die Spielzeit 2024/2025\n   - Drucksache TH-24/004 - zur Information\n\n2. Bericht des Intendanten\n   - mündlicher Bericht -\n\n3. Neufassung der Satzung über die Führung des Theater Freiburg als Eigenbetrieb\n   - Drucksache G-24/047 - beratend\n\n4. Bekanntgaben und Aktuelles\n\nH o r n Oberbürgermeister',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-The-80%7C1'}},
#  6: {'questions': ['Welche Auswirkungen hat die Absetzung des Tagesordnungspunktes zur Fortschreibung des Tourismuskonzepts auf die touristische Entwicklung Freiburgs?',
#    'Wie können sich Bürger über den neuen Zeitplan für die Diskussion des Tourismuskonzepts informieren?',
#    'Welche alternativen Möglichkeiten gibt es für Bürger, ihre Ideen zur touristischen Entwicklung Freiburgs einzubringen?'],
#   'document': {'f.extracted_text': 'Das Bürgermeisteramt der Stadt Freiburg im Breisgau - Dezernat I -\n\nFreiburg i. Br., 19.04.2024\nTel.: 0761/201-1110\nHerr Knobloch\n\nÄnderung der Tagesordnung zur 4. Sitzung des Gemeinderates am 23. April 2024\n\nMitglieder des Gemeinderates\n\nVon der Tagesordnung der Sitzung des Gemeinderates wird folgender öffentlicher Tagesordnungspunkt\n\nFortschreibung des Tourismuskonzepts für die Stadt Freiburg: Tourismuskonzept Freiburg 2024 (TK2)\n\n- Drucksache G-24/064 - beschließend\n\na b g e s e t z t.\n\nH o r n\nOberbürgermeister',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-GR-263%7C3'}},
#  7: {'questions': ['Wie wirken sich die Veränderungen in der Bearbeitung von Wohngeldanträgen konkret auf die Wartezeiten für Bürger aus?',
#    'Welche Verbesserungen können Bürger durch die organisatorischen Änderungen in der Abteilung Wohngeld und Wohnungssuche erwarten?',
#    'Wie plant die Stadt, die Ergebnisse der Mitarbeitendenbefragung zur Verbesserung des Bürgerservice zu nutzen?'],
#   'document': {'f.extracted_text': 'Mündlicher Bericht über AfS, Abteilung 7 Wohngeld und Wohnungssuche im Sozialausschuss am 18.04.2024\nSozialausschuss 18.04.2024 Seite 1\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\nFreiburg IM BREISGAU\n\nEntwicklung Fallzahlen (1)\n\nMonat | Antragseingänge insgesamt | Anzahl Antragseingänge im Vorjahr zum gleichen Zeitpunkt | Abweichung in % | Ergangene Bescheide | Ergangene Anschreiben wegen fehlenden Unterlagen\nOkt 23 | 1.129 | 665 | 70 | 859 | 1.150\nNov 23 | 1.071 | 739 | 45 | 984 | 1.282\nDez 23 | 944 | 742 | 27 | 703 | 737\nJan 24 | 1.039 | 1.573 | -33 | 839 | 909\nFeb 24 | 978 | 1007 | -2,8 | 637 | 767\nMrz 24 | Auswertung am Ende des Folgemonats, Statistik vorher nicht im Diwo abbildbar | 718 | 902\nApr 24 | laufender Monat | laufender Monat\n\nSozialausschuss 18.04.2024 Seite 2\n\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\n\nFreiburg IM BREISGAU\n\nEntwicklung Fallzahlen (2)\n\nFälle mit offener Entscheidung zum 1. Werktag des Monats\n\nMonat         Gesamt      Erstanträge      Folgeanträge\nJan 24        5.266       2.403            2.863\nFeb 24        5.395       2.381            3.014\nMrz 24        5.408       2.531            2.877\nApr 24        5.340       2.530            2.810\n\n⇒ Im April erstmals weniger Fälle mit offener Entscheidung als im Vormonat\n\nSozialausschuss 18.04.2024 Seite 3\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\nFreiburg IM BREISGAU\n\nOrganisatorische Veränderungen / Maßnahmen\n■ Aktuelle Stellensituation • bei den Kompetenzsachbearbeiter*innen Soll: 4,0 VZÄ Ist: 3,0 VZÄ zum 01.06. 0,6 VZÄ zusätzlich\nBei den Sachbearbeiter*innen Soll: 33,25 VZÄ Ist: 31,49 VZÄ in den nächsten Monaten fallen1,85 VZÄ weg (Kündigung, Arbeitsreduzierung, Mutterschutz)\n■ Einteilung der Abteilung in 2 Sachgebiete, bestehend aus insgesamt 5 Teams\n\nSozialausschuss 18.04.2024 Seite 4\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\n1 WY F re iburg IM BREISGAU\n\nOrganisatorische Veränderungen / Maßnahmen\n\nAbteilungssekretariat mit Servicebüro (2,0 VZA § 16i SGB II Umschüler_in) Qualitätsmanagement / Kompetenzsachbearbeitung (4,0 VZA) Sachgebiet I (Personalunion mit Abteilungsleitung) Sachgebiet II\n\nTeam 1 WBS / WoSu Team 2 Wohngeld Beginnt mit Publikum Team 3 Wohngeld Beginnt mit Sachbearbeitung Team 4 Wohngeld Beginnt mit Telefon Team 5 Wohngeld Beginnt mit Vertretung Publikum / Telefon\n\nRund 4,7 VZA Jeweils rund 7 VZA in der Sachbearbeitung\n\nSozialausschuss 18.04.2024 Seite 5 Amt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche Freiburg IM BREISGAU\n\nOrganisatorische Veränderungen / Maßnahmen\n■ Seit 01.02.2024 Anpassung der Telefonzeiten (Montag bis Freitag von 9.00 bis 12.00 Uhr) an die Zeiten des AfS\n■ Verbesserung der Online-Antragstellung\n■ Erneute eingeschränkte Erreichbarkeit vom 08. bis 19.04.2024\n\nSozialausschuss 18.04.2024 Seite 6\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\nIM BREISGAU\n\nPersonelle Veränderungen / Maßnahmen\n■ Aktuell Einarbeitung von neun Mitarbeiter*innen\n■ Zum 01.02.2024 haben 1,5 VZÄ „§ 16 i SGB II-Kräfte“ zur Unterstützung bei der Veraktung ihren Dienst aufgenommen. 0,5 VZÄ stehen noch zur Verfügung, aktuell laufen Gespräche wegen Arbeitszeitaufstockung\n■ Praktikum einer Umschülerin in der Zeit vom 19.03. – 06.09.2024 mit 25 Stunden die Woche\n\nSozialausschuss 18.04.2024 Seite 7\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\nFreiburg IM BREISGAU\n\nMitarbeitendenbefragung\n■ Umfrage in der Zeit vom 19.03. bis 05.04. in Zusammenarbeit mit dem HPA\nAbteilung Wohngeld und Wohnungssuche\n\nHerzlich Wilkommen zu unserer Umfrage!\nMit dieser Umfrage (ca. 5-10min.) wollen wir uns einen Überblick über verschiedene Aspekte innerhalb der Abteilung verschaffen, um darauf aufbauend die Zufriedenheit und die Zusammenarbeit zu verbessern. Deine Meinung ist uns sehr wichtig und liefert uns wertvolle Erkenntnisse für die Zukunft der Abteilung. Herzlichen Dank für Deine Teilnahme\n\nSozialausschuss 18.04.2024 Seite 8\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\nE 1 b WY rel urg 1M BREISGAU\n\nHerzlichen Dank für Ihre Aufmerksamkeit!\nSozialausschuss 18.04.2024 Seite 9\nAmt für Soziales Abteilung 7 – Wohngeld und Wohnungssuche\nFreiburg” IM BREISGAU',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-SO-199%7C3'}},
#  8: {'questions': ['Welche konkreten Auswirkungen hat die Investition von 500 Millionen Euro durch Pfizer auf den Arbeitsmarkt und die wirtschaftliche Entwicklung in Freiburg?',
#    'Wie profitieren Freiburger Bürger von der Wertschöpfung und den Arbeitsplätzen, die durch Pfizer in der Region geschaffen werden?',
#    'Welche Maßnahmen plant die Stadt Freiburg, um die Rahmenbedingungen für innovative Gesundheitsunternehmen wie Pfizer zu verbessern?'],
#   'document': {'f.extracted_text': 'PGS am Gesundheitsstandort Freiburg\nAusschuss Wirtschaft und Wissenschaft\nDr. Axel Glatz\nFreiburg, 8.4.24\nBreakthroughs that change patients’ lives\nPGS: Pfizer Global Supply\nConfidential 1\n\nVon 1962 bis heute, von Gödecke zu Pfizer\n\n2\n\nPGS Freiburg heute\n• Eine der weltweit modernsten Pharma-Fabriken zur Entwicklung und Herstellung hochwirksamer Arzneimittel\n• Größtes Pfizer-Werk zur Herstellung fester patentgeschützter Arzneiformen\n• 1,5 Mio. Packungen pro Arbeitstag in 2023\n• Produktion & Verpackung für 185 Länder: je 30% für Europa, USA und Asien sowie 10% für Südamerika/Afrika\n• Der Standort steht im weltweiten Wettbewerb mit anderen Pfizer Produktionsbetrieben\n• Ca. 500 Mio. € Investitionen in Freiburg in den letzten 10 Jahren\n• Mit ca. 2.200 Mitarbeitenden Freiburgs größter privater Arbeitgeber und einer der großen industriellen Gewerbesteuerzahler\nPGS: Pfizer Global Supply\n3\n\n„Zukunftswerk“ Freiburg: ein wichtiger Hebel für den lokalen Wohlstand\n• Pfizer generiert in Freiburg 174 Mio. € direkte Wertschöpfung.\n• Darüber hinaus werden deutschlandweit 81 Mio. € Wertschöpfung in Vorleistungsbranchen angestoßen – vielfach in Freiburg und der Region.\n• Weitere 34 Mio. € Wertschöpfung durch die Konsumausgaben der direkt und indirekt Beschäftigten.\nMit rund 2.000 Beschäftigten ist das Freiburger Werk ein wichtiger Arbeitgeber in der Region. • Rund 1 % der Beschäftigten in Freiburg sind bei\n- Rund 1% der Beschaftigten in Freiburg sind bei Pfizer beschaftigt.\nPfizer beschäftigt. Jeder Beschäftigte im „Zukunftswerk“ schafft einen weiteren Arbeitsplatz in anderen industriellen und Dienstleistungsbranchen.\nPfizer Freiburg: Bruttowertschöpfung, 2021 (in Mio. €)\n34  81  289  174  Direkte Wertschöpfung  Indirekte Wertschöpfung  Induzierte Wertschöpfung  Summe Wertschöpfung \nDurch Pfizer Freiburg deutschlandweit angestoBene * Gebaudenahe Dienstleistungen\nGebäudenahe Dienstleistungen\nTech. und naturwiss. Untersuchungen Rechts-, Steuer- und Unternehmensberatungen Großhandel\n14%  35%  12%  3%  81 Mio. €  4%  4%  4%  6%  6%  6%  6% \nPapierindustrie\nEnergieversorgung\nInformationsdienstleistungen\nGrundstücks- und Wohnungswesen Baugewerbe\nLagerei\nSonstige Branchen\nPfizer\nQuelle: Institut der deutschen Wirtschaft (2024): „Gesundheitsstandort Deutschland: Entwicklungen und Potenziale. Die Rolle der Pharmaindustrie für Wirtschaft und Gesellschaft“\n4\n\nVon der „Apotheke der Welt“ zum Arzneistoffimporteur Wir brauchen dringend Veränderungen\n\n• Die Produktion von Generika ist weitgehend aus Europa und Deutschland verschwunden. 2020 betrug die Produktion gerade noch 5% des Weltbedarfs. Generische Wirkstoffe kommen ganz überwiegend aus Asien\n\nIm Bereich patentgeschützter Arzneimittel beträgt der Europäische Anteil 54%, der Deutsche 12%. Im Jahr 2000 lag die Europäische Zahl bei 50%, Deutschland trug 25% bei\n\n• Dennoch tragen Pharma / Chemie mit 238,5 €Mrd. mehr zum Deutschen Außenhandel bei als die Automobilindustrie mit 209,4 €Mrd. und der Maschinenbau mit 194,4 €Mrd.\n\n• Ursächlich für die negative Tendenz sind Kosten, die steuerliche Situation, hohe Löhne und Abgaben und in den letzten Jahren eine proliferierende Gesetzesflut, verschlafene Digitalisierung und extrem langwierige Genehmigungsverfahren\n\n• PGS-Freiburg konnte sich der Entwicklung bisher entziehen. Dazu beigetragen haben Vorteile des deutschen Standortes – Rechtssicherheit, sozialer Friede, gute Infrastruktur, Konzentration von Geräte- Herstellern in Süddeutschland, Schweiz und Norditalien, gutes Ausbildungssystem und Schutz geistigen Eigentums. Nicht zuletzt auch durch Unterstützung der Stadt Freiburg.\n\nPfizer\n\n5 5\n\nWarum sollte ein innovatives Unternehmen der Gesundheitsindustrie in Freiburg produzieren? Botschaften für eine starke innovative Gesundheitsindustrie am Standort Freiburg\n\n1. Die Gesundheitsindustrie ist eine Schlüsselindustrie. Sie schafft nicht nur attraktive Arbeitsplätze und trägt maßgeblich zur Bruttowertschöpfung bei. Auch auf die Gesundheit und Arbeitsfähigkeit der Bevölkerung hat sie einen positiven Einfluss.\n\n2. Investitionsentscheidungen werden global und immer im Gesamtkontext getroffen. Sie sind nur möglich, wenn kontinuierlich an den richtigen Rahmenbedingungen gearbeitet wird und in einem funktionierenden Innovationsökosystem bestehend aus Grundlagenforschung, klinischer Forschung sowie Innovationen in der Produktion und in der Versorgung.\n\n3. Um in Deutschland Arzneimittel oder Medizinprodukte global wettbewerbsfähig zu produzieren, braucht es hohe Produktionsvolumen, Automation, ständige Innovation, Gewerbeflächen und hochqualifizierte Fachkräfte.\n\n4. Weniger bürokratische Anforderungen und einheitliche Standards entlasten unternehmerisches Handeln. Einfache und schnelle behördliche Verfahren sind ein zentraler Ansatz zur Stärkung der Standortattraktivität.\n\neS Pfizer\n\nConfidential 6\n\nDanke!\n\n—_— Pfizer\n\nBreakthroughs that change patients’ lives\n\nConfidential\n\n7',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-AWW-8%7C5'}},
#  9: {'questions': ['Welche konkreten Verbesserungen können Freiburger Bürger durch die Umsetzung des neuen Tourismuskonzepts in ihrem Alltag erwarten?',
#    'Wie werden die geplanten Schlüsselprojekte, wie z.B. die Verbindung von Altstadt und Stadtgarten, das Stadtbild und die Lebensqualität in Freiburg verändern?',
#    'Welche Möglichkeiten haben Bürger, sich an der Umsetzung der Schlüsselprojekte des Tourismuskonzepts zu beteiligen oder Feedback zu geben?'],
#   'document': {'f.extracted_text': 'FORTSCHREIBUNG\nTOURISMUSKONZEPT FREIBURG\n08. APRIL 2024\n\nRealizing Progress\nTourismuskonzept Freiburg 2024: Wo wir aktuell stehen\n\nDez. 2022 – Febr. 2023  Interner Kick-off • Vorgehen Timing • • Akteur*innen  März 2023  April/Mai  2023  • Desk Research • Persönliche  Gespräche  • Online-Umfrage  Bevölkerung  • Wirtschaftsfaktor   Tourismus  • Zahlen, Daten,   Fakten zu Angebot/  Nachfrage  • Quantitatives &   qualitatives Bench- marking  April/Mai  2023  Grobraster  Juni – Aug. 2023  • 12.06. Team- Workshop  Sept./Okt.  2023  • 15.06. Workshop  Stadtverwaltung • 04.07. Steuerungs- gruppensitzung  • 25.07., 26.07., 02.08.   Erarbeitungs- Workshops \n\n05.09. Steuerungs- gruppensitzung • Konkretisierung/\nGrobraster\n• 20.10. Steuerungs- gruppensitzung\n• Dez. 22: AR-Auftrag zur Fortschreibung TK\n• Ende Dez. 22 – Febr. 23: Ausschreibung & Agentur- auswahl (zusammen mit Steuerungsgruppe, gebildet aus Mitgliedern des Tourismusbeirats)\n\n\nRealizing Progress\nRahmen zur Umsetzung der Schlüsselprojekte: Die Übernachtungssteuer.\nAb 2024: 1 Mio. Euro pro Jahr für die FWTM.\n\nRealizing Progress\n\nNotwendige Grundlagen FWTM-intern\n\n• 01) Organisationsentwicklung der FWTM Neuausrichtung und Priorisierung der Aufgabenbereiche, Anpassung Abteilungsstrukturen\n• 02) Ausrichtung der Tourist Information Einbindung in touristische Sparte der FWTM, Produktentwicklung, Daten-/Informationsmanagement\n• 03) Neue Kennziffern und Messbarkeit Kennziffern für Arbeit der FWTM/Umsetzung Tourismuskonzept/Marke, Entwicklung Dashboard\n\nFREI —/ 4\n\nRealizing Progress\nSchlüsselprojekte Oberthema Freiburg- Gefühl mit den Unterthemen Kulinarik, Kultur, Workation\n\n• 04) Stadtteile/Ortschaften: Positionierung und Strukturierung\n• 05) Zielgruppen:\nBereitstellung relevanter Informationen\n• 06) Begegnungsstätten und Workation-\nInfrastruktur\n• 07) Markenkonforme Verbindung Altstadt – Stadtgarten\n• 08) Markenkonforme Verbindung Bahnhof – Altstadt\n• 09) Kulinarik: Food Märktle • 10) Workation: Location •\n11) Stadtteile/Ortschaften: Mit nachhaltigen Mobilitätskonzepten erlebbar machen\n• • •\n12) Neugestaltung Tourist Information\n13) Reisegruppen\n14) Kulinarik: Konzept\n15) Kultur: Veranstaltungen in der Nebensaison\n• 16) Workation: Event\n17) Standortmarketing: Recruiting-Maßnahmen\n18) Zielgruppen: (Multisensorische) Informationen im öffentlichen Raum\n\nRealizing Progress\nSchlüsselprojekte Profilierungsthema Natur & Aktiv\n\n19) Wasser: Bächle und Trinkbrunnen\n20) Wasser: Erlebnis an der Dreisam\n21) Attraktivierung und Ausflugsziel Tuniberg\n22) Attraktivierung und Ausflugsziel Schauinsland\n23) Aktiv: Radfahren: Marketing\n24) Aktiv: Mountainbike für alle: Governance\n25) Steigerung der Attraktivität nachhaltiger Mobilitätskonzepte für Aktiv-Reisende\n\nRealizing Progress\nSchlüsselprojekte Thema MICE & Geschäftsreisen\n• 26) Customer Journey (Kundenreise) bei MICE & Geschäftsreisen\n• 27) Markenkonforme Kommunikations- und Vertriebsstrategie\n• 28) Markenkonforme Gestaltung FWTM-eigener Locations\n• 29) Erarbeitung Code of Conduct (Verhaltenskodex)\n• 30) Schaffung von markenkonformen Produkt-Bausteinen\n\nRealizing Progress\nSchlüsselprojekte Basisthema Nachhaltigkeit\n• 31) Anbindung an Region durch KONUS\n• 32) Enabling (Befähigung) von Partnern im Bereich Nachhaltigkeit\n\nRealizing Progress\nSchlüsselprojekte Basisthema Digitalisierung/KI\n• 33) Touristische Datenoffensive Freiburg\n• 34) Enabling (Befähigung) von Partnern im Bereich Digitalisierung/KI\n• 35) Gästekarte als Zusatzangebot zu KONUS\n• 36) Verbesserung der Internet- Infrastruktur im öffentlichen Raum\n• 37) Bewegungsdaten\n\nRealizing Progress\nRelevante Schlüsselprojekte im Überblick Ausschuss für Wirtschaft und Wissenschaft\n\nOberthema Freiburg-Gefühl mit den Unterthemen Kulinarik, Kultur, Workation\n• 04) Stadtteile/Ortschaften: Positionierung und Strukturierung\n• 05) Zielgruppen: Bereitstellung relevanter Informationen\n• 10) Workation: Location\n• 11) Stadtteile/Ortschaften: Mit nachhaltigen Mobilitätskonzepten erlebbar machen\n• 16) Workation: Event\n• 17) Standortmarketing: Recruiting-Maßnahmen\n• 18) Zielgruppen: (Multisensorische) Informationen im öffentlichen Raum\n\n27) Markenkonforme Kommunikations- und Vertriebsstrategie\n28) Markenkonforme Gestaltung FWTM-eigener Locations\n29) Erarbeitung Code of Conduct\n30) Schaffung von markenkonformen Produkt-Bausteinen\n\nBasisthema Nachhaltigkeit\n32) Enabling von Partnern im Bereich Nachhaltigkeit\n\nBasisthema Digitalisierung/KI\n33) Touristische Datenoffensive Freiburg\n36) Verbesserung der Internet-Infrastruktur im öffentlichen Raum\n\nMICE & Geschäftsreisen\n26) Customer Journey bei MICE & Geschäftsreisen\n\nRealizing Progress\n\nAusblick: Wie geht es weiter?\n\n22.02.24 Kulturausschuss\n13.03.24 Mobilitätsausschuss\n18.03.24 Ausschuss für Umwelt und Klimaschutz\n22.03.24 Tourismusbeirat\n08.04.24 Ausschuss für Wirtschaft und Wissenschaft\n10.04.24 Bau-, Umlegungs- und Stadtentwicklungsausschuss\n15.04.24 Haupt- und Finanzausschuss\n23.04.24 Gemeinderat\n\nFREI\nBURG\n\nRealizing Progress\nTourismuskonzept Freiburg 2024: Wo wir aktuell stehen\nOkt. – Dez. 2023\n• Schreiben Tourismuskonzept\n• Grafiken & Kartographie\nDez. 23: Auftrag vom Aufsichtsrat an die FWTM, das TK2 in Abstimmung mit Stadtverwaltung/Tourismusbeirat in den Gemeinderat einzubringen\nJan 24: Tourismusbeirat verabschiedet TK2\n• mehrstufiger Abstimmungsprozess mit Stadtverwaltung\nFebr.-April 24: Gemeinderätliche Ausschüsse\nDez. 2023- April 2024\n23. April 2024\n• Gemeinderat\nTourismuskonzept und Maßnahmenplan\nAbstimmung Tourismuskonzept\nFreigabe Tourismuskonzept\n\nRealizing Progress\nZielstellung des Tourismuskonzepts 2024\nDer Tourismus in Freiburg ist stark und gut für die Stadt. Das Tourismuskonzept Freiburg 2024 hat das Ziel, die touristische Marke mit Leben zu füllen und in die Realität umzusetzen. Dies gilt sowohl für den Bereich Freizeit(Leisure)- als auch für den Geschäftsreisetourismus sowie für die Stadtentwicklung in touristischen Fragestellungen. In diesem Sinne geht die FWTW den nächsten Entwicklungsschritt und nimmt ihre Verantwortung für den Tourismus und in Richtung der Stadtgesellschaft in Freiburg verstärkt wahr.\n\nBedeutung der touristischen Marktsegmente\nAUFENTHALITSTAGE 16,651] MIO.\nUMSATZE 812,4 MIO. €\n\nOrao I Privatquartiere oN Gewerbliche Betriebe 78,1% S VFR Tagesreisen\n\nQuelle: dwif 2023\nSeite 7\nWirtschaftsfaktor Tourismus fiir die Stadt Freiburg 2022\nSOON SES JENS UTE MES\n\nDirekte Profiteure des Tourismus\n\nÜbernachtungsgäste 343,9 Mio. €\n\nTagesgäste inkl. VFR 468,5 Mio. €\n\nGESAMT 812,4 Mio. €\n\nGastgewerbe 386,8 Mio. €\n\nQuelle: dwif 2023\n\nSeite 8\n\nWirtschaftsfaktor Tourismus für die Stadt Freiburg 2022\n\nRealizing Progress\n\nUnsere strategischen Leitlinien\n\n• Marke Freiburg\n• Werte für Freiburg\n• Definierte Zielgruppen\n• Fokus auf Nebensaison\n• Fokus auf erdgebundene Reisen (Ausnahmen MICE & Geschäftsreisen, Gesundheit, Kooperation „Historic Highlights of Germany“)\n  MICE: Meetings, Incentives, Conferences & Events\n• Definierte Produktprinzipien\n• Definierte Themen\n\nRealizing Progress\nLeitziele für eine nachhaltige & stadtverträgliche Tourismus-Entwicklung in Freiburg\n• Tourismus ist ein integrativer Bestandteil der nachhaltigen Entwicklung der Stadt Freiburg und fördert einen verantwortungsvollen Umgang mit Ressourcen.\n• Touristisches Wachstum und Stadtverträglichkeit stehen in Einklang miteinander. Deshalb sind die Bedürfnisse von Gästen und Bevölkerung gleichermaßen relevant.\n• Freiburg ermöglicht es sowohl Gästen als auch der Bevölkerung, das typische Freiburg-Gefühl zu erleben. An allen Kontaktmomenten ist die touristische Marke Freiburgs erlebbar.\n• Es erfolgt eine Fokussierung auf qualitatives Wachstum im Tourismus im Sinne der definierten Zielgruppen sowie auf die Erhöhung der touristischen Wertschöpfung in Freiburg und der Region.\n• Durch die verstetigte Einbindung von touristischen Partnern und die Integration der Leistungsträgerinnen und Leistungsträger in die Tourismusarbeit sollen alle vom Tourismus profitieren können.\n• Freiburg orientiert sich auch im Tourismus an den Nachhaltigkeitszielen der Stadt Freiburg. Diese verfolgen einen ganzheitlichen Ansatz in Bezug auf ökologische, ökonomische und soziale Nachhaltigkeit.\n\nRealizing Progress\nThemenstruktur des Tourismuskonzepts\nOberthema FREIBURG-GEFUHL LEISURE Unterthema Unterthema Unterthema KULINARIK KULTUR WORKATION Basisthema DIGITALISIERUNG/KI Basisthema NACHHALTIGKEIT GESUNDHEIT\n\nRealizing Progress\nTourismuskonzept Freiburg 2024: Schlüsselprojekte und ihre Umsetzungsstufen',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-AWW-8%7C6'}},
#  10: {'questions': ['Welche konkreten Auswirkungen wird die geplante Einführung der Verpackungssteuer in Freiburg ab Juli 2025 für Bürger und Gastronomiebetriebe haben?',
#    'Wie sollen Freiburger Betriebe bei der Umstellung von Einweg- auf Mehrwegverpackungen unterstützt werden?',
#    'Welche messbaren Ziele verfolgt die Stadt Freiburg mit der Einführung der Verpackungssteuer hinsichtlich der Müllreduzierung im öffentlichen Raum?'],
#   'document': {'f.extracted_text': 'Drucksache G-24/043 Aktueller Sachstand und Verfahrensbeschluss zur Einführung der Verpackungssteuer\nAusschuss für Wirtschaft und Wissenschaft am 08. April 2024\n\nMichael Gedeon-Göbel\nStadtkämmerei\n\nDrucksache G-24/043 Aktueller Sachstand und Verfahrensbeschluss zur Einführung der Verpackungssteuer Seite 1 18.03.2024\n\nStadtkämmerei & Umweltschutzamt\n\nFreiburg IM BREISGAU\n\nDrucksache G-24/043\n\nAusgangslage Hohes Abfallaufkommen durch Einwegverpackungen für Speisen und Getränke zur Mitnahme (take-away)\n„Vermüllung“ des öffentlichen Raums Seit 2023 geltende gesetzliche Verpflichtung zur Bereitstellung von Mehrwegangeboten bei Einwegkunststofflebensmittelverpackungen hat nicht den gewünschten Erfolg gebracht\nAuch öffentliche Kampagnen (Augen auf!) führen nicht zu einer signifikanten Reduzierung der Müllmengen im öffentlichen Raum\nVerpackungssteuer könnte dieser Entwicklung entgegenwirken In Tübingen bereits erste Erfolge mit der Verpackungssteuer\n\nDrucksache G-24/043 Aktueller Sachstand und Verfahrensbeschluss zur Einführung der Verpackungssteuer Seite 2 18.03.2024\nStadtkämmerei & Umweltschutzamt\n\nDrucksache G-24/043\n\nSachstand in Freiburg\nPrüfauftrag der Verwaltung aus 2021 zur Einführung einer Verpackungssteuer bis zu einer Entscheidung des BVerwG zur Verpackungssteuer in Tübingen zurückgestellt\nIm Mai 2023 Bestätigung der Rechtmäßigkeit der Tübinger Verpackungssteuer\nUrteil des Bundesverfassungsgerichts steht noch aus\nSeitdem Prüfung der Einführung einer Verpackungssteuer und Informationsaustausch mit der Verwaltung in Tübingen\nMit vorliegender Drucksache Verfahrensvorschlag zur Einführung einer Verpackungssteuer in Freiburg\n\nDrucksache G-24/043 Aktueller Sachstand und Verfahrensbeschluss zur Einführung der Verpackungssteuer Seite 3 18.03.2024\nStadtkämmerei & Umweltschutzamt\nFreiburg IM BREISGAU\n\nDrucksache G-24/043\n\nRahmenbedingungen der Einführung einer Verpackungssteuer in Freiburg Ziel der Steuer ist nicht vorrangig die Erzielung von Einnahmen, sondern die Abfallreduzierung sowie die Eindämmung der Vermüllung des öffentlichen Raums\n\ndie Abfallreduzierung sowie die Eindämmung der Vermüllung des öffentlichen Raums\n\nDies wird idealerweise durch die Umstellung von Einwegverpackungen auf Mehrweg erreicht\n\nHierfür ist die Beratung der Betriebe und die gezielte Förderung von Mehrwegangeboten erforderlich\n\nBeratung der Betriebe, Umstellung in den Betrieben benötigen ausreichenden zeitlichen Vorlauf\n\nBereitstellung von Personal in der Verwaltung für diese Aufgaben erforderlich\n\nDrucksache G-24/043 Aktueller Sachstand und Verfahrensbeschluss zur Einführung der Verpackungssteuer Seite 4 18.03.2024\n\nStadtkämmerei & Umweltschutzamt\n\nFreiburg IM BREISGAU\n\nDrucksache G-24/043\n\nVerfahrensvorschlag Beschluss zur Einführung der Verpackungssteuer in Freiburg mit\nSatzungsbeschluss im Herbst 2024\nEinführung der Verpackungssteuer zum 01.07.2025 Nach Satzungsbeschluss Beginn der Information und Beratung der Betriebe\nund Austausch mit den verschiedenen Interessenverbänden\n\nDrucksache G-24/043 Aktueller Sachstand und Verfahrensbeschluss zur Einführung der Verpackungssteuer Seite 5 18.03.2024\nStadtkämmerei & Umweltschutzamt\nFreiburg IM BREISGAU',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-AWW-8%7C7'}},
#  11: {'questions': ['Welche konkreten Maßnahmen plant der Migrant_innenbeirat, um die Wahlbeteiligung bei der anstehenden Wahl 2025 zu erhöhen?',
#    'Wie will die Stadt Freiburg auf die hohe Betroffenheit von häuslicher Gewalt und Zwangsheirat unter Frauen mit Migrations- und Fluchterfahrung reagieren?',
#    'Welche spezifischen Verbesserungen im Gesundheitssystem plant die Stadt Freiburg, um sprachliche Barrieren für Migrant_innen abzubauen?'],
#   'document': {'f.extracted_text': 'Migrant_innenbeirat der Stadt Freiburg\n\nMigrant_innenbeirats-Sitzung (öffentlich) Mittwoch, den 6. März 2024\n\nProtokoll\n\nBeginn: 18:00 Uhr\nEnde: 20:10 Uhr\nModeration: Yrgalem Abreha\nProtokoll: Başar Alabay\nAddendum: Präsentation „Frauen, Gesundheit und Rechte“ als PDF\nAnwesende und Gäste: Separate Beiblätter (intern)\n\nBegrüßung\n\n- Die Co-Vorsitzende Frau Hamman eröffnet um 18:10 Uhr die Sitzung. Es werden als Referentinnen begrüßt: Frau Häußler und Frau Mathis, Ärztinnen im Freiburger Zentrum für Frauen mit Genitalbeschneidung der Universitäts-Frauenklinik Freiburg.\n- Frau Hamman erläutert, dass der Migrant_innenbeirat (MB) beschlossen hat, seine öffentlichen Sitzungen aus dem Rathaus heraus „in die Bevölkerung“ zu tragen. Daher findet diese nun im Saal der Kirchengemeinde St. Albert in Betzenhausen statt. 13 Gäste finden ihren Weg zur Veranstaltung.\n- Frau Hamman weist darauf hin, dass 2025 die Wahl des neuen MB ansteht. Sie wirbt für den MB, seine Funktionen und Aufgaben. Wer sich engagieren möchte, solle sich aufstellen lassen und mitmachen. Alle Wahlberechtigten sind aufgerufen, von ihrem Recht auch Gebrauch zu machen.\n- Nach einer kurzen Vorstellung der Frauenkommission durch die Co-Vorsitzende, übergibt diese das Wort an Frau Abreha, die im weiteren Verlauf die Sitzung moderiert.\n\nTOP 1: Frauengesundheit und Frauenrechte\n\n- Frau Abreha rekurriert auf eine Umfrage zwischen Frauen mit Flucht- und Migrationserfahrung mit Fokus auf Partizipation in Politik und Gesellschaft, welche die Frauenkommission 2021–2022 gemacht hatte. Teilhabe und Schutz waren hierbei wichtige Themen. Unter anderem wurde erfragt, ob es Betroffenheit von häuslicher Gewalt gibt – 36 % bejahten dies. Von Zwangsheirat waren 47 % betroffen. Dies zeige, wie wichtig das Thema Frauenrechte ist. Daher wollte die Frauenkommission die zwei Felder Frauenrechte und Frauengesundheit miteinander verbinden. In der Vergangenheit hatte es bereits eine Kooperation mit der Universitäts-Frauenklinik zum Thema Genitalbeschneidung (FGM_C) gegeben.\n- Frau Häußler und Frau Mathis – beide an der Frauenklinik zuständig für die FGM_C-Sprechstunde – reißen kurz an, welche Überforderung im\n\n1\n\nGesundheitssystem existiert. Insbesondere sprachliche Hürden erschwerten vieles. Flyer hülfen hierbei nicht unbedingt, wenn Sprachkompetenzen fehlten. - Es folgt ein Vortrag samt Präsentation, der interaktiv die Bereiche Gesundheit und Rechte zusammenfasst.\n- Die folgenden Inhalte des Vortrags sind der (nach Rücksprache mit den Ärztinnen) beigelegten Präsentation entnehmbar!\n\nFragen, Diskussion und Austausch\n\n- Die folgende Diskussion wird von zwei Freiburger Vertreterinnen der Parteien SPD und Bündnis 90/Die Grünen, die als Gäste anwesend sind, bereichert. - Hinweis durchs bei der Sitzung ebenfalls vertretene Refugium Freiburg auf die Gesundheitslots_innen. Es gebe durchaus Infoveranstaltungen in Freiburg und Breisgau-Hochschwarzwald – es sei jedoch besonders schwierig, an Männer heranzukommen. Man komme gerne überall hin, wohin man eingeladen werde. Direkt in der LEA werden jedoch keine Veranstaltungen angeboten.\n- Es wird vorgeschlagen, dass der MB die Site des Refugiums mit Hinweis auf die Gesundheitslots_innen verlinkt.\n- Dem Integrationsmanagement fehlten oft die Informationen, was was wichtig ist. Könnte es nicht Info-Veranstaltungen geben?\n- Das FGZ erstellt auch Gutachten, auch für Asylverfahren beim BAMF. - Beim Thema FGM_C wird noch einmal unterstrichen, dass oftmals kulturelle oder religiöse Hintergründe mitspielten. Es gebe auch Kinderbeschneidung in Deutschland oder bei Reisen ins Heimatland. Hier sei der Kontakt mit Kinderkliniken wichtig.\n- Es wird noch einmal dezidiert auf den hohen Prozentsatz von depressiven Frauen mit Migrationsgeschichte hingewiesen: 24 %. Hier stehen als Anlaufstellen Frauenhorizonte oder das Refugium zur Verfügung. Es gibt Frauengruppen, doch nie ausreichend Personen, die sich der Thematik annehmen könnten. Überwiegend werden therapeutische Einzelbehandlungen statt Gruppenangebote gewünscht. Hinweis auf das PAMF-Netzwerk.\n- Es kommt die Nachfrage, ob eigentlich an der Uniklinik das Thema Diskriminierung benannt und angegangen wird. Hierzu gebe es bisher keine Fortbildungen.\n- Das Zentrum für Psychiatrie in Emmendingen hält eine Transkulturelle Ambulanz vor.\n- Es kommt die Frage auf, wo man sich beschweren müsste, wenn man Leistungen selbst zahlen müsste: Bei den Krankenkassen, in der Regel online.\n- Das Thema Dolmetschen wird angesprochen. In der Uniklinik werden manchmal ad hoc Videodolmetschende hinzugezogen. Hinweis auf den Dolmetschendenpool.\n- Es kommt die generelle Frage auf, wo man derzeit eigentlich im Bereich Frauenrechten stünde. Haben Frauenthemen auf politischer Ebene ausreichend Lobbyarbeit? Kann man damit zufrieden sein? Es wird festgestellt, dass dies auch weiterhin ein politisches Thema bleibe – zum Beispiel bei Wahlen (und selbstverständlich danach).\n- Wie kann es eine sinnvolle Präventionsarbeit geben, die sich mit den Strukturen von Macht und Gewalt auseinandersetzt?\n- Es folgt eine angeregte und vielseitige Diskussion zum Feminismus – was er erreicht hat, wo er steht, welche Bedeutung er hat; und was noch alles zu bewältigen ist.\n\n2',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-Mib-4%7C1'}},
#  12: {'questions': ['Welche Auswirkungen hat die Einführung von E-Scooter-Sharing-Systemen auf die Mobilität und den öffentlichen Raum in Freiburg?',
#    'Wie werden die Abstellverbotszonen für E-Scooter in der Freiburger Innenstadt durchgesetzt und kontrolliert?',
#    'Welche Daten müssen die E-Scooter-Anbieter der Stadt Freiburg zur Verfügung stellen und wie werden diese genutzt?'],
#   'document': {'f.extracted_text': 'TOP 1: Miet-E-Scooter in Freiburg\n\nBehindertenbeirat am 05.03.2024 Seite 1\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg IM BREISGAU\n\nMietsysteme für Elektro-Tretroller in Freiburg\n\nYoio ■ ■ ■\nYourCar GmbH Göttingen seit 12/2020 200 Scooter\n\nROXY ■ Roxy Mobility ■ seit 04/2022 ■ 150 Scooter\n\nZEUS ■ ■ ■\nZEUS Scooters GmbH seit 04/2023 150 Scooter\n\nWeitere Anfragen seit 2020: Tier Mobility; Lime Bike; Bird; Wind; Voi Scooters; Spin; Flash / Circ\n\nBehindertenbeirat am 05.03.2024 Seite 2\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\nFreiburg” IM BREISGAU\n\nInhalte der Vereinbarungen zwischen Stadt und Anbietern\n\nAufbau\n1. Präambel\n2. Fahrzeuge\n3. Flottengröße, Betriebsgebiet und Verteilung der Roller\n4. Flottenmanagement, Wartung, Betrieb\n5. Nutzung der E-Tretroller\n6. Auf- und Abstellen der E-Tretroller\n7. Kontakt\n8. Datensammlung, -verwendung und -schutz\n9. Unterstützung von Seiten der Stadt Freiburg\n10. Vorzulegende Nachweise\n\nVereinbarung Uber die Etablierung eines E-Tretroller-Leihsystems in der Stadt Freiburg im Breisgau zwischen Stadt Freiburg im Breisgau verteten durch den Oberbirgerreister Marth W. W. Hom, vertreten durch das Garten- und Tiefbauamt, Fehrenbachallee 12, 79106 Freiburg ~nachflgend ,STADT FREIBURG" genannt— und E-Tretroller-Anbieter (Firmenname)\n\nE-Scooter Absteliverbot Innenstadt Zone 1 E-Scooter verboten eee | Frebure®\n\nBehindertenbeirat am 05.03.2024 Seite 3\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg” IM BREISGAU',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-BehB-15%7C3'}},
#  13: {'questions': ['Wie plant die Stadt Freiburg, die Abschlussquote von Leistungsvereinbarungen in den verschiedenen Bereichen der Eingliederungshilfe zu erhöhen?',
#    'Welche konkreten Maßnahmen werden ergriffen, um insbesondere im Bereich der Tagesstruktur und Wohnformen für Minderjährige die Zahl der abgeschlossenen Vereinbarungen zu steigern?',
#    'Wie wirkt sich der aktuelle Stand der Leistungsvereinbarungen auf die Verfügbarkeit von Plätzen in den verschiedenen Einrichtungen für Menschen mit Behinderungen aus?'],
#   'document': {'f.extracted_text': 'Auswertung LRV — 31.12.2023\n\nProzentualer Anteil aufgeforderter Angebote/Plätze in Klammern\n\nLeistungstyp Angebote Davon: aufgefordert Davon: abgeschlossen Plätze Davon: aufgefordert Davon: abgeschlossen\nWfbM 566 539 (95,2 %) 273 (48,2 %) 29.986 28.218 (94,1 %) 14.763 (49,2 %)\nErwachsene - Sonstige Tagesstruktur 935 787 (84,2 %) 306 (32,7 %) 16.118 13.297 (82,5 %) 5.403 (33,5 %)\nErwachsene - Besondere Wohnform* 1.749 1.457 (83,3 %) 513 (29,3 %) 24.593 20.844 (84,8 %) 7.857 (31,9 %)\nMinderjährige – Tagesstruktur 215 106 (49,3 %) 9 (4,2 %) 8.043 3.367 (41,9 %) 336 (4,2 %)\nMinderjährige – Besondere Wohnform 180 109 (60,6 %) 21 (11,7 %) 3.197 1.574 (49,2 %) 370 (11,6 %)\nSonstige Angebote zur sozialen Teilhabe** n.v. n.v. n.v. - - -',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-BehB-15%7C5'}},
#  14: {'questions': ['Welche konkreten Verbesserungen der Barrierefreiheit können Bürger_innen an den umgebauten Bushaltestellen in Freiburg erwarten?',
#    'Wie wirkt sich der Umbau der Haltestelle Bissierstraße auf die Nutzbarkeit des öffentlichen Nahverkehrs für Menschen mit Behinderungen aus?',
#    'Welche zusätzlichen Maßnahmen plant die Stadt Freiburg, um die Sicherheit und Mobilität von blinden und sehbehinderten Menschen im Straßenverkehr zu verbessern?'],
#   'document': {'f.extracted_text': 'TOP 2: Verwendung der Haushaltsmittel DHH 2023/24 für Barrierefreiheit im öffentlichen Raum\n\nBehindertenbeirat am 05.03.2024 Seite 1\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg\n\nVerwendung der Haushaltsmittel DHH 2023/24 „Barrierefreiheit im öffentlichen Raum“\n\nVerfügbare Haushaltsmittel in 2023\n\nFinanzhaushalt \n■ Planansatz 2023 im DHH 2023/2024 350.000 EUR \n■ Übertragene Mittel aus 2022 354.000 EUR \n\nErgebnishaushalt \n■ Planansatz 2023 Bissierstraße im DHH 2023/2024 200.000 EUR \n\nVerfügbare Mittel für Barrierefreiheit im öffentlichen Raum gesamt (2023) \n■ Summe Planansätze Finanz- und Ergebnishaushalt sowie Mittelübertragung Finanzhaushalt 905.000 EUR\n\nBehindertenbeirat am 05.03.2024 Seite 2\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\n© Ww Freiburg IM BREISGAU\n\nHaltestelle Bissierstraße (Betzenhausen), Linien 3, 10, 22, 36\n\nBehindertenbeirat am 05.03.2024 Seite 11\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg IM BREISGAU\n\nHaltestelle Bissierstraße (Betzenhausen), Linien 3, 10, 22, 36\n\nBehindertenbeirat am 05.03.2024 Seite 12\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg IM BREISGAU\n\nVerwendung der Haushaltsmittel DHH 2023/24 „Barrierefreiheit im öffentlichen Raum“\n\nBarrierefreie Umgestaltung von Bushaltestellen 2023\n■ Dorfstraße (Günterstal, Umstieg Stadtbahn – Bus, Bussteige) 620.000 EUR\n■ Munzinger Straße (Haid, Umstieg Stadtbahn – Bus, Bussteige) 160.000 EUR\n■ Marienmattenweg (Haslach) 100.000 EUR\n■ Laufener Straße (Weingarten) 160.000 EUR\n■ Schneeburgstraße (St. Georgen), im Rahmen der Umgestaltung Uffhauser Platz 105.000 EUR\n■ Restzahlungen für Haltestellenumbau 2022: Löwen (Littenw.) + Keidel-Bad 80.000 EUR 15.000 EUR\n\nZur Ausführung in 2024/2025:\n■ Bissierstraße (Planung und Mittelbindung für Umbau in 2024) 215.000 EUR\n■ Planungsmittel für Haltestellenumbau in 2024/2025: Kirchbergstraße (Lehen), 190.000 EUR\nTiengener Straße (St. Georgen), Immentalstraße (Herdern), Hochmeisterstraße (Herdern), Ebnet-Rathaus (Ebnet), Pädagogische Hochschule (Littenweiler), Stadttheater 25.000 EUR\n\nZusatzeinrichtungen für Blinde und Sehbehinderte an Lichtsignalanlagen 25.000 EUR\n■ LSA Schwarzwaldstraße / Flaunserstraße 15.000 EUR\n■ LSA Schauinslandstraße / Wonnhalde / Haltestelle Wonnhalde 10.000 EUR\n\nAusgaben und Mittelbindungen im Ergebnishaushalt 105.000 EUR\n\nAusgaben und Mittelbindungen insgesamt 965.000 EUR\n\nBehindertenbeirat am 05.03.2024 Seite 3\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\n© Ww Freiburg IM BREISGAU\n\nHaltestelle Dorfstraße (Günterstal), Linien 2 und 21\n\nBehindertenbeirat am 05.03.2024 Seite 4\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg” IM BREISGAU\n\nHaltestelle Dorfstraße (Günterstal), Linien 2 und 21\nar: a: ee AAT "6\nBehindertenbeirat am 05.03.2024 Seite 5\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\nFreiburg” IM BREISGAU\n\nHaltestelle Munzinger Straße (Haid), diverse Linien\n\nBehindertenbeirat am 05.03.2024 Seite 6\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg IM BREISGAU\n\nHaltestelle Munzinger Straße (Haid), diverse Linien\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nBehindertenbeirat am 05.03.2024 Seite 7\n\nFreiburg IM BREISGAU\n\nHaltestelle Marienmattenweg (Haslach), Linie 14\n\nBehindertenbeirat am 05.03.2024 Seite 8\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg IM BREISGAU\n\nHaltestelle Schneeburgstraße (St. Georgen), Linie 11\n\nBehindertenbeirat am 05.03.2024 Seite 9\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\nFreiburg IM BREISGAU\n\nUffhauser Platz / Schneeburgstraße (St. Georgen)\n\nBehindertenbeirat am 05.03.2024 Seite 10\n\nGarten- und Tiefbauamt Hendrik Schmitt-Nagel\n\n© Freiburg IM BREISGAU',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-BehB-15%7C4'}},
#  15: {'questions': ["Welche Auswirkungen hat die Absetzung des Tagesordnungspunkts zur Entschließung über den 'Radikalenerlass' auf die Bürger von Freiburg?",
#    "Wie können sich Bürger über den aktuellen Stand der Diskussion zum Thema 'Berufsverbote' in Baden-Württemberg informieren?"],
#   'document': {'f.extracted_text': 'Das Bürgermeisteramt\nder Stadt\nFreiburg im Breisgau\n- Dezernat I -\n\nFreiburg i. Br., 24.01.2024\nTel.: 0761/201-1110\nHerr Knobloch\n\nÄnderung der Tagesordnung zur\n1. Sitzung des Gemeinderates am\n30. Januar 2024\n\nMitglieder des Gemeinderates\n\nVon der Tagesordnung der Sitzung des Gemeinderates wird folgender öffentlicher Tagesordnungspunkt 5\n\nEntschließung des Freiburger Gemeinderats zum sog. "Radikalenerlass" bzw. zu den "Berufsverboten" in Baden-Württemberg (Interfraktioneller Antrag nach § 34 Gemeindeordnung vom 10.10.2023)\n\n- Drucksache G-24/042 - beschließend\n\na b g e s e t z t.\n\nH o r n\nOberbürgermeister',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-GR-260%7C3'}},
#  16: {'questions': ['Welche konkreten Maßnahmen plant die Stadt Freiburg, um das friedliche Zusammenleben von Menschen aus 160 Nationen weiter zu fördern?',
#    'Wie können Bürger in Freiburg aktiv zur Verteidigung von Demokratie und Freiheit beitragen, wie es in der Erklärung gefordert wird?'],
#   'document': {'f.extracted_text': 'Erklärung für ein weltoffenes Freiburg „Nie wieder ist jetzt“\n\nSeit Jahren versuchen Rechtsextremist:innen unsere Gesellschaft zu spalten, indem sie hasserfüllte Parolen gegen Migrant:innen, Menschen mit Behinderungen und sozial Benachteiligte verbreiten. Ziel ihrer menschenfeindlichen Ideologie ist es, das friedliche Zusammenleben zu stören und unsere Demokratie abzuschaffen.\n\nAllen Menschen muss bewusst sein, dass in einem rechtsextremen System niemand mehr vor Verleumdung, Denunziation und Verfolgung sicher sein wird. Die Forderung nach sogenannter Remigration für Menschen mit Migrationshintergrund, selbst mit deutscher Staatsbürgerschaft, sind mehr als ein Warnsignal an die gesamte Gesellschaft. Sie legen die zutiefst menschen- und verfassungsfeindliche Ideologie solcher Positionen offen.\n\nAls demokratische Fraktionen im Freiburger Gemeinderat treten wir geschlossen dieser Hetze entgegen, um zu verhindern, dass sich dunkle Kapitel unserer Geschichte wiederholen. Wir stehen für Weltoffenheit, Toleranz und eine vielfältige Gesellschaft. Unabhängig von Herkunft, Geschlecht, Religion, sexueller Orientierung oder Gesundheitszustand vertreten wir die Anliegen aller Bürger:innen.\n\nIn Freiburg leben Menschen aus 160 Nationen friedlich zusammen, und wir werden dieses friedliche Miteinander weiter fördern. Die Mobilisierung von über 35.000 Menschen in Freiburg für Demokratie und Solidarität gibt uns Mut. Es liegt an uns, Freiheit und Demokratie zu verteidigen, sei es in der Familie, im Beruf oder im sozialen Umfeld.\n\nDemokratiefeindlichkeit, Rechtsextremismus, Rassismus, Antisemitismus und jede andere Form von gruppenbezogener Menschenfeindlichkeit haben in unserer Stadt keinen Platz. Freiburg bleibt weltoffen, steht für Solidarität und ein soziales Miteinander, wehrt sich und bleibt standhaft gegen Rechtsextremismus!\n\nFreiburg, den 30. Januar 2024\n\nBündnis 90/ Die Grünen SPD/Kulturliste JUPI FW\n\nEine Stadt für Alle CDU FDP/BfF FL',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-GR-260%7C4'}},
#  17: {'questions': ['Welche Auswirkungen haben die Änderungen im Beteiligungsbericht 2023 auf die Dienstleistungen der städtischen Gesellschaften für die Bürger?',
#    'Wie können Bürger Einsicht in die Nachhaltigkeitsberichte der städtischen Beteiligungen erhalten und welche Verbesserungen sind dadurch zu erwarten?'],
#   'document': {'f.extracted_text': 'Das Bürgermeisteramt\nder Stadt\nFreiburg im Breisgau\n- Dezernat I -\n\nFreiburg i. Br., 17.01.2024\nTel.: 0761/201-1110\nHerr Knobloch\n\n1. Sitzung des Haupt- und Finanzausschusses (Sondersitzung zum Beteiligungsbericht)\nErgebnismitteilung\n\nAn die Mitglieder des Gemeinderates\n\nDer Haupt- und Finanzausschuss hat in der öffentlichen Sitzung am 15. Januar 2024 folgende Punkte behandelt:\n\nTOP 1\n(verbundene Debatte mit TOP 2)\n\nBeteiligungsbericht 2023 mit Jahresabschlüssen 2022, Wirtschaftsplänen 2024, mittelfristigen Finanz- und Investitionsplänen 2024 bis 2028, Wirtschaftsplanänderungen 2023\n\nDie Tagesordnungspunkte 1 und 2 werden als verbundene Debatte geführt, da die städtischen Gesellschaften in ihren Präsentationen auch das Thema „Nachhaltigkeit“ behandeln.\n\nDie Anlage 1 der Drucksache G-24/006 ist wie folgt zu berichtigen:\n\nAuf S. 23 ist der Beginn des Kapitels 3 „Planungsprämissen“ zu streichen, da dieses nicht Bestandteil der Leistungs- und Finanzziele der Gemeinnützigen Freiburger Qualifizierungs- und Beschäftigungsgesellschaft mbH (f.q.b.) ist. Es handelt sich um eine redaktionelle Klarstellung.\n\nVortrag: BM Breiter (Drucksachen G-24/006 und G-24/001)\n\nStR Dengler, StR Dr. Schüle und StR Dr. Gröger kommen zur Sitzung.\n\nWortmeldungen: Herr Backhaus, Nachhaltigkeitsmanagement\nFrau Dr. Szablewska, Freiburger Stadtbau GmbH\n\nBM Prof. Dr. Haag und StR Schwarz kommen zur Sitzung.\n\nHerr Dr. Müller, Freiburger Stadtbau GmbH\n\nStR Saleh kommt zur Sitzung.\n\n- 2 -\nStR Mohlberg\nStR Dr. Schüle\nOB Horn\nBM Breiter\nFrau Blümle, f.q.b. gGmbH\nEBM von Kirchbach\nHerr Broglin, Abfallwirtschaft und Stadtreinigung Freiburg GmbH\nHerr Benz, Freiburger Verkehrs AG\nHerr Bartosch, Freiburger Verkehrs AG\nFrau Schonhard, Stadtwerke Freiburg GmbH\n\nStRin Schwer kommt zur Sitzung\n\nHerr Hellebrand, badenova AG & Co. KG\nStRin Viethen\nFrau Böhme, Freiburg Wirtschaft Touristik und Messe GmbH (digital zugeschaltet)\n\nStR Sumbert verlässt die Sitzung.\n\nHerr Dr. von Zahn, Umweltschutzamt\n\nStR Glück verlässt die Sitzung.\n\nStRin Federer\nStR Dr. Gröger\nStRin Dr. Jenkner\nStR Dengler\n\nStR Dr. Gröger verlässt die Sitzung.\n\nDie Fraktionen sind auf Vorschlag von OB Horn damit einverstanden, dass auf die Anwesenheit der Geschäftsführung und Vorstände der städtischen Gesellschaften in der Sitzung des Gemeinderates am 30.01.2024 verzichtet wird.\n\nErgebnis\n\nDer Haupt- und Finanzausschuss verweist diesen Tagesordnungspunkt zur abschließenden Entscheidung in den Gemeinderat.\n\n- 3 -\nTOP 2\n(verbundene Debatte mit TOP 1)\nDritte Nachhaltigkeitsberichterstattung städtischer Beteiligungen\nErgebnis\nDer Haupt- und Finanzausschuss verweist diesen Tagesordnungspunkt zur abschließenden Entscheidung in den Gemeinderat.\nIm Auftrag\nF o l k e r t s',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2024-HFA-79%7C11'}},
#  18: {'questions': ['Welche konkreten Verbesserungen können Eltern von Kindern mit Behinderungen durch den Einsatz von Heilpädagog_innen in ausgewählten Kitas erwarten?',
#    'Wie wirkt sich die geplante Evaluation zum Einsatz von Heilpädagog_innen auf die zukünftige Gestaltung inklusiver Kitas in Freiburg aus?',
#    'Welche Vorteile bietet das neue Konzept der strukturellen Inklusionsleistungen für Familien mit Kindern ohne Behinderungen?'],
#   'document': {'f.extracted_text': 'Strukturelle Inklusionsleistungen in Kindertageseinrichtungen\nDez II / ASK\nSitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nHintergrund\n\nBeschluss des Aktionsplans für ein inklusives Freiburg 2015/2016 vom Gemeinderat am 17.05.2015\n1 von 92 Maßnahmen: Konzept zur Umsetzung von Inklusion in den Freiburger Kitas\nModellprojekte von 2015 bis 2017\n\nÄmterübergreifende Arbeitsgruppe seit 2018\nKonzeptionelle Vorüberlegungen mit dem Ziel, das Recht auf soziale Teilhabe in Kitas besser und effizienter umzusetzen\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nEinsatz der bewilligten Heilpädagog_innen im Rahmen des Doppelhaushaltes 2023/2024\n\nStädt. Kinder- und Familienzentrum Sprungbrett (Betzenhausen)\n\nStädt. Kita Tausendfühler (Haslach)\n\nStädt. Kinder und Familienzentrum Landwasser (Landwasser)\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nVorschlag der Verwaltung zur Umsetzung des Fraktionsauftrages\n\nEinsatz der Personalmittel i.H.v. 230.000 EUR (ca. 3 VZÄ) in 3 Kitas in freier Trägerschaft\n\nEinstellung von Heilpädagog_innen, die Teil des Teams einer Kita werden (strukturelle Maßnahme)\n\nAuswahl des Planungsraums nach Anzahl bewilligter Eingliederungshilfen\n\nAuswahl der Kita im Rahmen eines Interessensbekundungsverfahrens\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nVorschlag der Verwaltung zur Umsetzung des Fraktionsauftrages\n\nBegleitung durch eine vom Steuerungsgremium eingerichtete Begleitgruppe mit Expert_innen aus Behindertenbeirat, SPZ, FFB und bei Bedarf weitere, Leistungserbringer_innen, Vertretungen der freien Träger\n\nEvaluation zum Einsatz von Heilpädagog_innen als strukturellen Bestandteil des Kitateams mittels Befragungen der beteiligten Akteur_innen, Fokusgruppen mit Fallmanager_innen sowie best practice Analyse\n\nweitere Empfehlungen zum Ausbau inklusiver Kitas in ca. 2 Jahren\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nZiele einer inklusiven Kita\n\nRahmenbedingungen in den Kitas im Sinne eines strukturellen Angebotes so verbessern, dass der bestehende Anspruch auf Unterstützungsleistungen verlässlich, gesichert, in guter Qualität erfolgt\n\nInklusionsfachkräfte begleiten alle Kinder und unterstützen insbesondere Kinder mit Teilhabebedarfen\n\nDie inklusive Haltung in der Kita ist selbstverständlich\n\nQualitätsentwicklung und Qualitätssicherung der Inklusionsleistungen in Kitas\n\nDie Handlungsfähigkeit des Kitapersonals wird gestärkt und bleibt erhalten\n\nQualitätssteigerung durch fachliche Standards, Verlässlichkeit und Kontinuität\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nZiele einer inklusiven Kita\n\nGrößere Zufriedenheit bei allen Beteiligten\nFrühzeitige Bedarfserkennung und Stärkung der Prävention\nReduzierung von individuellen Anträgen auf Eingliederungshilfe durch Schaffung struktureller Lösungen\nSchaffung eines wohnortnahen Angebotes für Eltern\nMittel- bis langfristige Ausweitung auf weitere Planungsräume\nMittelfristige Stabilisierung des fiskalischen Aufwands bei weiter steigendem Bedarf an Inklusionsleistungen\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nPlanungsräume\n\nPlanungs- raum Gebiete Anzahl Fälle pro Kita 2022 (265)\n3 Haslach-Egerten, Haslach-Gartenstadt, Haslach-Schildacker 39 (15 %)\n4 Weingarten, Haslach-Haid 31 (12 %)\n10 Landwasser, Lehen 28 ( 8 %)\n11 Brühl-Beurbarung, Brühl-Güterbahnhof, Brühl-Industriegebiet 22\n16 Oberau, Mittel- /Oberwiehre 15\n17 Ebnet, Kappel, Littenweiler, Waldsee 15\n6 Herdern-Nord, Zähringen 14\n1 Munzingen, Opfingen, Tiengen, Waltershofen 13\nMooswald 12\n8 9 2 18 12\nMooswald Betzenhausen Mundenhof, Rieselfeld Günterstal, Unterwiehre Alt-Stühlinger, Stühlinger-Eschholz 12 12 11 11 10\n15 Altstadt-Mitte, Altstadt-Ring 9\n13 14 7 5\nVauban Sankt Georgen Hochdorf Neuburg, Herden-Süd 7 7 6 3\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nVielen Dank für Ihre Aufmerksamkeit!\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nHintergrund\n\nGesetzesänderung im SGB VIII, § 22a Abs. 4\nKinder mit Behinderungen und Kinder ohne Behinderungen sollen gemeinsam gefördert werden. Die besonderen Bedürfnisse von Kindern mit Behinderungen und von Kindern, die von Behinderung bedroht sind, sind zu berücksichtigen (früher, sofern der Rahmen dies zulässt)\n\nFraktionsanfrage vom 15.02.2023\nKonzept zur Inklusionsbegleitung in Kitas erstellen und umsetzen\nIst-Zustand und Bedarfsanalyse nach Planungsräumen ermitteln\nZiel: In jedem Planungsraum ein Angebot für Eltern von Kindern mit Inklusionsbedarf, mind. 1 Kita mit mind. einer heilpäd. Fachkraft\nFinanzielle Mittel: 230.000 EUR zur Umsetzung für Personal in Kitas, Planungsmittel in Höhe von 50.000 EUR für die wissenschaftliche Begleitung und konzeptionelle Weiterentwicklung von inklusiven Angeboten\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nAktuelle Entwicklung\n\nSteigender Bedarf an Anzahl und Intensität von Inklusionsleistungen\n\nErforderliche kurzfristige, insbesondere umfangreiche Bedarfe können zunehmend nicht zuverlässig erbracht werden\n\nMehrere, zum Teil wechselnde leistungserbringende Personen sind in einer Kita tätig, arbeiten mit einzelnen Kindern\n\nKonzeptionelle Abstimmungen sind in der Regel nur schwer oder nicht ausreichend mangels Zeit möglich\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nAktuelle Entwicklung\n\nKinder mit umfangreichen Teilhabebedarfen erhalten nicht immer die erforderliche Förderung\n\nEinzelne Fachkräfte und Teams sind zunehmend überfordert\n\nTransferaufwand für Eingliederungsleistungen im Bereich Kitas lag im Jahr 2022 bei ca. 3,7 Mio. EUR\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nWarum strukturelle Lösungen in Kitas?\n\nGesetzlicher Auftrag - Pflicht zur inklusiven Gestaltung von Kitas (KJSG)\nMit der Änderung des § 22a Abs. 4 SGB VIII am 09.06.2021 besteht die Pflicht, Kinder mit und ohne Behinderungen in Gruppen grundsätzlich gemeinsam zu fördern (davor: sofern der Hilfebedarf dies zulässt).\nDie besonderen Bedürfnisse von Kindern mit Behinderungen und von Kindern, die von Behinderung bedroht sind, sind zu berücksichtigen. Das zielt auf die Schaffung struktureller Maßnahmen.\nIm Fokus stehen Kinder mit besonderen Bedarfen im Kitasystem, nicht mehr das einzelne Kind!\nMit der Strukturlösung wird der inklusive Gedanke umgesetzt.\n\nStrukturelle eingebundene Inklusionsfachkräfte\nbegleiten alle Kinder gruppenbezogen und gruppenübergreifend\nKennen Kinder und Eltern, dadurch ist der Zugang für Eltern niederschwellig\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nWarum strukturelle Lösungen in Kitas?\n\nEffekte\nReduzierung von Antragsverfahren:\nVerzicht auf Antrag ist bürgerfreundlich und niederschwellig\nSenkt die Aufwendungen für die Eingliederungshilfe\nSchnelle Reaktion auf Entwicklungsschritte der Kinder und Bedarfsschwankungen mit verlässlichen Bezugspersonen\nVerkürzung von Wartezeiten für Kinder mit Förderbedarf\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nWarum Heilpädagog_innen?\n\nHeilpädagog_innen\nHandeln präventiv durch das frühzeitige Beobachten und Begleiten von Kindern mit Entwicklungsrisiken\nhaben spezifisches Fachwissen, von dem alle Kinder, deren Familien und Fachkräfte profitieren\nberaten Eltern professionell und niederschwellig\nsind täglich in der Kita kennen alle Eltern und binden sie frühzeitig ein\nerbringen Inklusionsleistungen im Rahmen der Eingliederungshilfe, die auf ungehinderte Teilhabe abzielen\nführen Fortbildungen, Beratungen, Qualifizierungen und Teamentwicklungen mit dem Index für Inklusion für alle Fachkräfte in der Kita durch\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nWarum Heilpädagog_innen?\n\nHeilpädagog_innen\nbegleiten und beraten Personen fachlich, die begleitende Hilfen durchführen\nunterstützen pädagogische Fachkräfte bei der Vorbereitung und in Elterngesprächen\nsetzen den Inklusionsauftrag auf Grundlage der Stärken und Strukturen in der Kita um\narbeiten konzeptionell und entwickeln die Qualität der pädagogischen Arbeit weiter\nhandeln in der Arbeit mit Kindern frühzeitig, so dass Eingliederungshilfen teilweise nicht (mehr) benötigt werden oder die Hilfe kürzer und weniger intensiv erforderlich ist\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nBeispiel Kita am Seepark\n\nIm Jahr 2020 wurden für 26 Kinder heilpädagogische Interventionen durchgeführt, die 3 Jahre von den Heilpädagog_innen und Fachkräften der Kita am Seepark begleitet wurden.\n\nEine Überprüfung mit den Kostenträger_innen AKI und AfS ergab:\nBei 13 Kindern (50%) wären vermutlich ohne die strukturellen Hilfen individuelle Eingliederungshilfen bewilligt worden\nBei 3 Kindern reichten die vorliegenden Informationen für eine Entscheidung nicht aus\nBei 10 Kindern wurde ein Hilfebedarf anerkannt, der anderweitig gedeckt werden musste (z.B. Hilfe zur Erziehung)\n\nDez II / ASK Sitzung des Beirates für Menschen mit Behinderung am 21.11.2023',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2023-BehB-14%7C6'}},
#  19: {'questions': ['Wie wirken sich die abgeschlossenen Leistungsvereinbarungen in den verschiedenen Bereichen der Eingliederungshilfe auf die Verfügbarkeit von Plätzen für Betroffene aus?',
#    'Welche Verbesserungen können Nutzer von Werkstätten für behinderte Menschen (WfbM) durch die neuen Leistungsvereinbarungen erwarten?'],
#   'document': {'f.extracted_text': 'Auswertung LRV – 31.10.2023\n\nProzentualer Anteil aufgeforderter Angebote/Plätze in Klammern\n\nLeistungstyp Angebote Davon: aufgefordert Davon: abge- schlossen Plätze Davon: aufgefordert Davon: abge- schlossen\nWfbM 584 505 (86,5 %) 166 (28,4 %) 29.975 26.804 (89,4 %) 9.068 (30,3 %)\nErwachsene - Sonstige Tagesstruktur 939 729 (77,6 %) 157 (16,7 %) 16.187 12.169 (75,2 %) 2.736 (16,9 %)\nErwachsene - Besondere Wohnform* 1.771 1.355 (76,5 %) 294 (16,6 %) 24.367 19.126 (78,5 %) 4.430 (18,2 %)\nMinderjährige – Tagesstruktur 281 116 (41,3 %) 12 (4,3 %) 10.198 3.689 (36,2 %) 231 (2,3 %)\nMinderjährige – Besondere Wohnform 78 52 (66,7 %) 0 819 447 (54,6 %) 0\nn.v. n.v. n.v. - - -\n\nSonstige Angebote zur sozialen Teilhabe**\n\n14. November 2023 Seite 1',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2023-BehB-14%7C7'}},
#  20: {'questions': ['Welche konkreten Maßnahmen sind für das Quartier Kleineschholz geplant, um es zu einem inklusiven und gemischten Wohngebiet zu machen?',
#    'Wie wird sichergestellt, dass im Quartier Kleineschholz bezahlbarer Wohnraum entsteht?',
#    'Welche Auswirkungen hat das Konzeptvergabeverfahren für Kleineschholz auf die Vielfalt der Wohnprojekte und Bewohnerstruktur?',
#    'Wie können sich Bürger am Entwicklungsprozess des Quartiers Kleineschholz beteiligen?'],
#   'document': {'f.extracted_text': 'Quartier Kleineschholz\nSitzung des Beirates für Menschen mit Behinderung am 21.11.2023\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 1\nProjektgruppe Kleineschholz Dezernat I und V\nFreiburg IM BREISGAU\n\nKleineschholz - Modellquartier\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 2\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nFreiburg IM BREISGAU\n\nQuartier Kleineschholz – Ziele und Leitbild\n\nBeispiele:\n• Belebte Erdgeschoss-Zonen mit Café, Bäckerei, etc.,\n• Kleingewerbe wie Fahrradwerkstatt, Friseur, Buchladen etc.,\n• Co-Working-Flächen,\n• Optionsräume für vielfältige Nutzungen (für Künstler_innen, Kulturschaffende, Quartiersarbeit), ...\nInnovativer und bezahlbarer Wohnraum\nQuartier der kurzen Wege\nGemischtes, inklusives Quartier für Alle\nKlima- und umweltfreundliches Quartier\nArchitektonisch ansprechende Gestaltung\nGemeinschaftliche Grün- und Freiflächen\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 11\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nFreiburg IM BREISGAU\n\nKonzeptvergabeverfahren – Ablauf im Überblick\n\n1. Phase -Bewerbung- 2. Phase -Auswahl und Reservierung-\nÖffentliche Info-Veranstaltung Bewerbung Auswahl Vergabe Reservierung Vertrag \n• Vorstellung Vermarktungs -konzept • Einreichen der Bewerbung (Konzeptidee) • Bewerbungsgespräche • Entscheidung des GR über Grundstücks- reservierung (= Zuschlag) • Sichtung der Bewerbungen • Beurteilung durch Bewertungsgremium (= Vergabevorschlag) • Statusgespräche • Vermessung der Baugrundstücke • Notarielle Beurkundung Grundstücksverträge\n• Planung bis Genehmigungsplanung • Finanzierungsnachweis • Beginn Hochbau\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 12\nProjektgruppe Kleineschholz Dezernat I und V\nE re iburgY IM BREISGAU\n\nZeitplan / weiteres Vorgehen\n\nDezember 2023 – Beschluss B-Plan – Beschluss Vermarktungskonzept 2024/2025 – Reservierungsphase – Vermessung – Verträge 2023 2024 2025 November 2023 Beginn Erschließung ca. 1. Quartal 2024 Beginn Vermarktung inkl. Vermarktungs- auftakt (öffentliche Veranstaltung) ca. Spätjahr 2025 Beginn Hochbau\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 13\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nFreiburg IM BREISGAU\n\nSchluss\n\nHerzlichen Dank für Ihre Aufmerksamkeit\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 14\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nE 1 b Ww rel urg IM BREISGAU\n\nLage und Bebaubarkeit\n\n, Flachenbilanz: Ctungsboreich: Urbane Gebietes = iN > =\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 3\n\nProjektgruppe Kleineschholz Dezernat I und V\n\n1M BREISGAU E re iburgY\n\nQuartier Kleineschholz - Baufelder\n\n■ ca. 16 Einzelprojekte\n\n■ MU 9: Marktgrundstück\nInklusionsbetrieb → Wünschenswert, dass mind. 30-50 % der Mitarbeitenden, Personen mit einer Behinderung sind\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 4\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nFreiburg IM BREISGAU\n\nQuartier Kleineschholz – Ziele und Leitbild\n\nZiel: Urbanes, nachhaltiges, inklusives, gemischt genutztes und lebendiges Wohnquartier mit Fokus auf Gemeinwohlorientierung\n\nInnovativer und bezahlbarer Wohnraum\nQuartier der kurzen Wege\nGemischtes, inklusives Quartier für Alle\nKlima- und umweltfreundliches Quartier\nArchitektonisch ansprechende Gestaltung\nGemeinschaftliche Grün- und Freiflächen\nAktive Liegenschaftspolitik der Stadt\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 5\nProjektgruppe Kleineschholz Dezernat I und V\nFreiburg IM BREISGAU\n\nKonzeptvergabe Kleineschholz\n\nDie Vergabe der Grundstücke erfolgt in einem „Wettbewerb der Ideen“.\n\n➢ 2-stufiges Vergabeverfahren mit niederschwelligen Anforderungen in der Bewerbungsphase\n\n➢ Ganzheitliche, beschreibende Bewertung der Konzeptideen in der Auswahlphase (anhand Leitbild – kein Punktesystem)\n\n➢ Innovative und kreative soziale und inklusive und ökologische Projekte\n\n➢ Unterstützung der Wirtschaftlichkeit des Projekts durch hohe Flexibilität in Planung und Realisierung\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 6\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nE 1 b N rel urg IM BREISGAU\n\nKonzeptverfahren Kleineschholz\nZugangsvoraussetzungen\n|\nGemeinwohl- orientierung\nmind. 50 %-Anteil geförderter Mietwohnungsbau\nErbbaurecht (Ablöse) / erbbaurechtsähnliches Wiederkaufsrecht\nWettbewerb der Ideen/Bewertung der Konzepte\nSitzung des Behindertenbeirats am 21.11.2023 Seite 7\nProjektgruppe Kleineschholz Dezernat I und V\nFre iburgY IM BREISGAU\n\nBewertung und Auswahl der Konzeptideen\nBewertung der Konzepte über ein vergleichendes Abwägen anhand festgelegter Bewertungskriterien\n➢ Qualität der Projektidee\n➢ Realisierungsverlässlichkeit\n➢ Beitrag für das Quartier und/oder Stadtgebiet\n→ Leitbild mit Zielen für die Quartiersentwicklung und Vermarktung\nSitzung des Behindertenbeirats am 21.11.2023 Seite 8\nProjektgruppe Kleineschholz Dezernat I und V\nE 1 b N rel urg IM BREISGAU\n\nQuartier Kleineschholz – Ziele und Leitbild\n\nBeispiele:\n• Mehrgenerationenwohnen,\n• Clusterwohnungen,\n• WGs für Senior_innen oder Menschen mit Behinderungen,\n• große Familien-Wohnungen,\n• Wohnungen / WGs für Alleinerziehende,\n• Kleinstwohnungen,\n• modulares Bauen, ...\n\nInnovativer und bezahlbarer Wohnraum\n\nQuartier der kurzen Wege\n\nGemischtes, inklusives Quartier für Alle\n\nKlima- und umweltfreundliches Quartier\n\nArchitektonisch ansprechende Gestaltung\n\nGemeinschaftliche Grün- und Freiflächen\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 9\n\nProjektgruppe Kleineschholz Dezernat I und V\n\nFreiburg IM BREISGAU\n\nQuartier Kleineschholz – Ziele und Leitbild\n\nInnovativer und bezahlbarer Wohnraum\nBeispiele:\n• Gemeinschaftsräume, Werkstätten, Wohnhöfe als Treffpunkte,\n• Einzelwohnen mit Assistenz, Pflege-Wohngruppe,\n• Wohnen für Geflüchtete,\n• generationenübergreifende Angebote mit gegenseitiger Hilfestellung, …\n\nKlima- und umweltfreundliches Quartier\nArchitektonisch ansprechende Gestaltung\nGemeinschaftliche Grün- und Freiflächen\nGemischtes, inklusives Quartier für Alle\nQuartier der kurzen Wege\n\nSitzung des Behindertenbeirats am 21.11.2023 Seite 10\nProjektgruppe Kleineschholz Dezernat I und V\nFreiburg IM BREISGAU Fre iburgY',
#    'f.id': 'https://ris.freiburg.de/oparl/file/sia%7C2023-BehB-14%7C8'}},
#  21: {'questions': ['Welche finanziellen Konsequenzen hat der Verkauf des städtischen Grundstücks für den Haushalt der Stadt Freiburg?',
#    'Wie wirkt sich der Wegfall der Mieteinnahmen von 13.780,80 EUR pro Jahr auf zukünftige Haushaltsplanungen aus?',
#    'Welche Pläne hat die Stadt für die Verwendung des außerordentlichen Ertrags von 411.417,40 EUR aus dem Grundstücksverkauf?'],
#   'document': {'f.extracted_text': 'Anlage 1 zur DRUCKSACHE HFA-24/007\n\n- Finanzielle Auswirkungen -\n\nBereich: Doppelhaushalt / Stadt Freiburg\nTeilhaushalt / Teilbudget: THH 22 - Amt für Liegenschaften und Wohnungswesen\nProduktgruppe(n): 1133\nSeite: 196 im Haushaltsplan des DHH 2023/2024\n\nErgebnishaushalt 2024\n- Mindererträge Nicht bezifferbar*\n\nFinanzhaushalt (für investive Maßnahmen) 2024\nEinzahlungen 842.080,00 EUR\nAuszahlungen - 00,00 EUR\nFinanzierungsüberschuss 842.080,00 EUR\n\nAußerordentlicher Ertrag 2024 411.417,40 EUR\n\nAuswirkung Folgejahre\nErgebnishaushalt 2025 ff\n- Mindererträge - 13.780,80 EUR\nNettoressourcenbedarf - 13.780,80 EUR\n\nEingestellte Mittel im HHPl./MiFi: ☐ in voller Höhe ☒ teilweise ☐ keine\n\nErläuterungen: Durch den Verkauf von Grundstücken ergibt sich ein außerordentlicher Ertrag, sofern der erzielte Kaufpreis höher ist als der im Anlagevermögen ausgewiesene Buchwert. Der außerordentliche Ertrag aus dem städtischen Grundstück Flst.Nr. 2551/6 errechnet sich aus dem Verkaufspreis in Höhe von 842.080,00 EUR abzüglich des Restbuchwertes von 430.662,60 EUR. Dies ergibt einen Betrag von 411.417,40 EUR.\n\nDas Grundstück ist bislang vermietet. Mit Vollzug des Kaufvertrages wird das Mietverhältnis von der Stadt gekündigt. Damit entfallen die Mieteinnahmen von monatlich 1.148,40 EUR, das sind 13.780,80 EUR p.a.\n\n*Für das Jahr 2024 sind die Minuserträge (wegfallende Mieteinnahmen) nicht bezifferbar, da dies vom Zeitablauf des weiteren Verfahrens abhängt. Aktuell steht noch nicht fest, zu welchem Zeitpunkt der Kaufvertrag beurkundet und zu welchem Zeitpunkt der Mietvertrag letztlich gekündigt wird.',
#    'f.id': 'https://ris.freiburg.de/oparl/file/vla%7C5430711100006%7C1'}},
#  22: {'questions': ['Welche Auswirkungen haben die neuen Bauvorschriften für das Quartier Kleineschholz auf die Gestaltung von Wohngebäuden und öffentlichen Räumen?',
#    'Wie werden die Vorgaben zu Fahrradstellplätzen die Mobilität im neuen Quartier beeinflussen?',
#    'Welche Konsequenzen hat die Begrenzung der Kfz-Stellplätze auf 0,3 pro Wohneinheit für Bewohner und Besucher des Quartiers?',
#    'Wie wirken sich die Vorschriften zu Werbeanlagen auf das Erscheinungsbild und die Geschäftsmöglichkeiten im Quartier aus?'],
#   'document': {'f.extracted_text': 'Anlage 2 zur DRUCKSACHE G-23/196\n\nSatzung der Stadt Freiburg i. Br.\n\nBebauungsplan mit örtlichen Bauvorschriften „Quartier Kleineschholz“, Plan-Nr. 5-121\n\nAufgrund des § 4 Abs. 1 der Gemeindeordnung für Baden-Württemberg in der Fassung vom 24. Juli 2000 (GBI. S. 581, ber. S. 698), zuletzt geändert durch Artikel 4 des Gesetzes vom 27. Juni 2023 (GBl. S. 229, 231), des § 10 Abs. 1 des Baugesetzbuches in der Fassung der Bekanntmachung vom 3. November 2017 (BGBl. I S. 3634), zuletzt geändert durch Artikel 1 des Gesetzes vom 28. Juli 2023 (BGBl. 2023 I Nr. 221) und des § 74 der Landesbauordnung für Baden-Württemberg in der Fassung vom 5. März 2010 (GBl. S. 358, ber. S. 416) zuletzt geändert durch Gesetz vom 13. Juni 2023 (GBl. S. 170) hat der Gemeinderat in seiner Sitzung am (Datum des Satzungsbeschlusses) folgende Satzung beschlossen.\n\n§ 1 Bebauungsplan und Geltungsbereich\n\nFür den Bereich\n\nder Grundstücke Flst.Nrn. 6388/2 (Fuß-/Radweg), 6388/3 (Kleineschholz), 6388/4 (Kleineschholz), 6388/5 (früher Agentur für Arbeit) und 6400 (Sundgauallee) sowie für Teilflächen der Grundstücke Flst.Nrn. 3059/25 (Lehener Straße), 6391 (Sundgauallee), 6479 (Lehener Wanner), 6394 (Rathaus im Stühlinger), 8324/2 (Bahn), 6479/2 (Lehener Wanner) und 6491/4 (Eschholzpark)\n\nbegrenzt\n\nim Norden durch die Lehener Straße,\nim Osten durch das Grundstück der Agentur für Arbeit sowie die Bereiche des Rathauses im Stühlinger und der städtischen Kindertagesstätte,\nim Süden durch die Trasse der Stadtbahn, einen Teil der Sundgauallee einschließlich des Brückenbauwerks und den Eschholzpark und im Westen durch die Güterbahnlinie Gundelfingen-Leutersberg,\n\nim Stadtteil Stühlinger,\n\nwird ein verbindlicher Bauleitplan nach § 10 Abs. 1 BauGB bestehend aus\n\n1. der Planzeichnung vom (Datum des Satzungsbeschlusses)\n2. den Textlichen Festsetzungen vom (Datum des Satzungsbeschlusses)\n\nBezeichnung: Bebauungsplan mit örtlichen Bauvorschriften "Quartier Kleineschholz", Plan-Nr. 5-121,\n\nbeschlossen.\n\nSatzung, Plan-Nr. 5-121\n\nMaßgebend für die räumliche Abgrenzung des Geltungsbereichs ist bei einem Widerspruch zwischen dem Textteil der Beschreibung des Geltungsbereichs und der Planzeichnung die Planzeichnung vom (Datum des Satzungsbeschlusses).\n\n§ 2 Örtliche Bauvorschriften\n\nZusätzlich werden nach § 74 LBO für das in § 1 bezeichnete Gebiet folgende örtliche Bauvorschriften erlassen:\n\nHinweis: Weitergehende Hinweise und Empfehlungen sind dem Gestaltungsleitfaden, der im Vergabeverfahren für die zu bebauenden Grundstücke zu berücksichtigen ist, zu entnehmen. Dieser wird den Bauherr_innen im Rahmen der Vermarktung zur Verfügung gestellt werden und dient als Orientierungshilfe bei der Gestaltung der Projekte.\n\n1. Anforderungen an die äußere Gestaltung baulicher Anlagen (§ 74 Abs. 1 Nr. 1 LBO)\n1.1. In den Urbanen Gebieten sind sowohl auf den Hauptgebäuden als auch auf den Nebenanlagen ausschließlich Flachdächer zulässig. Als Flachdach gilt eine Dachneigung zwischen 0 und maximal 5 Grad.\n1.2. Kunststoffverkleidungen an Gebäudefassaden sowie fluoreszierende, glänzende oder spiegelnde Oberflächen für Dächer und Fassaden unzulässig. Hierunter fallen auch Dächer und Fassaden aus unbeschichteten Metallen wie Kupfer, Zink oder Blei und deren Legierungen. Metalldacheindeckungen sind nur zulässig als Abdeckung der oberen Wandabschlüsse (Brüstungen, Attikas) und sofern diese aus beschichteten Blechen, Aluminium oder Edelstahl hergestellt sind.\n1.3. Glänzende oder spiegelnde Oberflächen und Materialien sind nur ausnahmsweise zulässig, wenn sie der Nutzung von Sonnenenergie dienen. Sie sind, sofern sie nicht auf Dachflächen angebracht werden, möglichst blendarm auszurichten.\n\n2. Werbeanlagen (§ 74 Abs. 1 Nr. 2 LBO)\n2.1. Werbeanlagen und Schilder sind nur an der Stätte der eigenen Leistung am Hauptgebäude bis zur Oberkante der Brüstung des 1. Obergeschosses und nur bis zu einer Gesamtfläche von 2 m² zulässig.\n2.2. Für sich stehende Schriftzüge sind in Einzelbuchstaben auszuführen und am Gebäude anzubringen. Die maximale Größe der Einzelbuchstaben beträgt 0,50 m in Höhe und Breite.\n\n- 2 -\n\nSatzung, Plan-Nr. 5-121\n\n2.3. Werbeanlagen in Form von Fremdwerbung, Lauflicht- und Wechselanlagen, Laserwerbung, Fahnenwerbung, großflächige Werbetafeln ≥ 2 m² und Anlagen, die dem Anschlag von Plakaten und anderen werbewirksamen Einrichtungen dienen, Booster (Lichtwerbung am Himmel) und Werbeanlagen auf dem Dach sind im Plangebiet unzulässig.\n\n2.4. Das Anbringen von Werbeanlagen auf Fensterflächen ist unzulässig.\n\nAnforderungen an die Gestaltung, Bepflanzung und Nutzung der unbebauten Flächen der bebauten Grundstücke (§ 74 Abs. 1 Nr. 3 LBO)\n\n3.1. Die unbebauten Flächen der bebauten Grundstücke, die nicht als Wege oder Zuwegungen zu den Rampen der Fahrrad-Tiefgaragen dienen, sind gärtnerisch als Grünflächen zu gestalten und dauerhaft zu erhalten. Die Gestaltung von Grünflächen unter Verwendung von Kunstrasen ist unzulässig.\n\n3.2. Großflächig mit Steinen, Kies, Schotter oder sonstigen vergleichbaren losen Materialschüttungen bedeckte Flächen, in welchen diese (Steinen, Kies, Schotter oder sonstigen vergleichbaren losen Materialschüttungen) das hauptsächliche Gestaltungsmittel sind und Pflanzen nicht oder nur in geringer Zahl vorkommen (Schottergärten), sind unzulässig.\n\nHinweise: Das gesetzlich geltende Verbot von Schottergärten nach § 21a Naturschutzgesetz BW 2020 ist zu beachten.\n\n4. Einfriedungen (§ 74 Abs. 1 Nr. 3 LBO)\n\nEinfriedungen zu den öffentlichen Verkehrsflächen, zu den privaten Erschließungswegen sowie den Gemeinschaftsfreiflächen innerhalb der Wohnhöfe sind nur in Form von lebenden Hecken aus heimischen Laubgehölzen und begrünten Mauern oder Zäunen aus Holz oder Metall, die an der Seite zur öffentlichen Verkehrsfläche bzw. zum privaten Erschließungsweg oder zu den Gemeinschaftsfreiflächen mit Hecken hinter- bzw. bepflanzt sind, zulässig. Einfriedungen aus Kunststoff sind unzulässig. Ihre Höhe ist auf maximal 0,8 m über dem jeweiligen angrenzenden Straßenniveau begrenzt. Dies gilt auch für Einfriedungen von Versickerungsmulden.\n\nEmpfehlung: Aus gestalterischen Gründen wird empfohlen, öffentliche und private Versickerungsmulden durch Bepflanzung und Geländemodellierung so zu gestalten, dass eine Einfriedung entfallen kann.\n\n- 3 -\n\nSatzung, Plan-Nr. 5-121\n\n5. Antennen, Mobilfunk- und Richtfunkanlagen (§ 74 Abs. 1 Nr. 4 LBO)\n5.1. Je Gebäude ist jeweils maximal eine Außenantenne zulässig. Diese ist nur auf dem Dach und den dem öffentlichen Raum abgewandten Gebäudeseiten zulässig.\n5.2. Mobil- und Richtfunkanlagen müssen einen Abstand von mindestens 2,00 m vom Rand des Gebäudes einhalten und dürfen nicht höher als 2,50 m sein.\n5.3. Satellitenantennen müssen die gleiche Farbe wie die dahinter liegende Dach- oder Wandfläche aufweisen.\n\n6. Kfz-Stellplätze (§ 74 Abs. 2 Nr. 1, Nr. 3, Nr. 4 und Nr. 5 LBO)\n6.1. Die Verpflichtung zur Herstellung notwendiger Kfz-Stellplätze für Wohnungen wird auf 0,3 Stellplätze / Wohneinheit eingeschränkt. Eine Herstellung von mehr als 0,3 Stellplätze / Wohneinheit ist untersagt.\n6.2. Errechnet sich bei der Ermittlung der Zahl der notwendigen Stellplätze eine Bruchzahl, ist nach allgemeinem mathematischem Grundsatz auf ganze Zahlen auf- bzw. abzurunden.\n6.3. Die notwendigen Stellplätze sind in der Quartiersgarage im Urbanen Gebiet MU 5 herzustellen.\n\n7. Abstellplätze für Fahrräder (§ 74 Abs. 2 Nr. 6 LBO)\n7.1. Die Anzahl notwendiger Fahrradstellplätze (Stellplätze für Fahrräder und Sonderräder) für Wohnungen nach § 37 Absatz 2 Satz 2 LBO ermittelt sich nach der Gesamtwohnfläche. Je 30 m² Wohnfläche ist ein Fahrradstellplatz herzustellen.\n7.2. Je angefangene 20 Fahrradabstellplätze ist zusätzlich eine Fläche für Fahrradanhänger bzw. Sonder- oder Lastenfahrräder von 2,6 m² zuzüglich der erforderlichen Fahrgassen und Rangierflächen vorzusehen.\n7.3. Zusätzlich sind Fahrradabstellplätze für Besucher_innen in ausreichender Zahl zu errichten. Bei Wohngebäuden sind mindestens vier Fahrradabstellplätze für Besucher_innen zielnah zum jeweiligen Zugang herzustellen.\n7.4. Zusätzlich sind Fahrradabstellplätze für Mitarbeitende in den Urbanen Gebieten in ausreichender Zahl zu errichten. Diese können ausnahmsweise für die Urbanen Gebiete MU 3, MU 4, MU 8, MU 9, MU 10, MU 11, MU 14 und MU 15 in der Quartiersgarage im Urbanen Gebiet MU 5 angeordnet werden.\n\n- 4 -\n\nSatzung, Plan-Nr. 5-121\n\nHinweise zu Nrn. 7.3 und 7.4: Fahrradstellplätze bei anderen Anlagen als Wohnungen sind entsprechend der VwV Stellplätze zielnah zu der jeweiligen Nutzung bzw. zu dem jeweiligen Zugang anzuordnen. Für die Ermittlung der Zahl der notwendigen Fahrradstellplätze bei anderen Anlagen als Wohnungen wird auf die Richtzahlen für Fahrrad-Stellplätze in der VwV Stellplätze verwiesen.\n\n7.5. Die Mindestmaße eines Fahrradabstellplatzes nach Nrn. 7.1, 7.3 und 7.4 betragen 2,00 m x 0,50 m zuzüglich der erforderlichen Fahrgassen und Rangierflächen.\n\n7.6. Die Fahrgassenbreite zwischen den Fahrradabstellplätzen beträgt mindestens 1,80 m.\n\n7.7. Die Fahrradabstellplätze sind mit fest verankerten Einstell- oder Anlehnvorrichtungen auszustatten, an denen die Fahrradrahmen angeschlossen werden können.\n\n7.8. Einfache Vorderradhalter sind unzulässig. Höchstens 50 % der nachzuweisenden Fahrradstellplätze können in Doppelstockparksystemen hergestellt werden.\n\n7.9. Die Abstellplätze nach Nrn. 7.1, 7.2 und 7.4 sind auf dem Baugrundstück in einem wettergeschützten und abschließbaren Raum zu errichten, der auf dem Niveau der öffentlichen Verkehrsfläche oder im Untergeschoss liegt, sofern die Erschließung über Rampen erfolgt, die eine Breite von mindestens 2,30 m und eine Neigung von maximal 15 % aufweisen.\n\nHinweise zu Nr. 7.9.: Die Lage und Zuordnung notwendiger Rampen nach Nr. 7.7 ist frei wählbar. Eine gemeinsame Nutzung für zwei benachbart liegende Grundstücke / Gebäude ist zulässig. Um eine komfortable Benutzung der Rampen zu den Fahrradgaragen für alle Begegnungsfälle zu gewährleisten, wird eine Rampenbreite von mindestens 2,50 m empfohlen.\n\n7.10. Zwischen der öffentlichen Verkehrsfläche und den Abstellplätzen dürfen maximal zwei Türen oder Tore mit einer Breite von mindestens 1,05 m liegen; Diese Türen müssen mühelos passierbar sein, z. B. indem sie über automatische Öffnungstasten verfügen.\n\nHinweis: Die Zahl und Beschaffenheit der nach dieser Vorschrift herzustellenden Fahrradabstellplätze ist im Baugenehmigungsverfahren in den Bauantragsunterlagen (Lageplan, Untergeschoss und Freiflächengestaltungsplan) nachzuweisen.\n\n- 5 -\n\nSatzung, Plan-Nr. 5-121\n\n§ 3 Ordnungswidrigkeiten\n\n(1) Ordnungswidrig im Sinne von § 75 Abs. 3 Nr. 2 LBO handelt, wer vorsätzlich oder fahrlässig den in § 2 genannten Vorschriften zuwiderhandelt.\n\n(2) Ordnungswidrigkeiten können nach § 75 Abs. 4 LBO in Verbindung mit § 17 Abs. 1 und 2 OWiG mit einer Geldbuße geahndet werden.\n\n§ 4 Inkrafttreten\n\nDie Satzung tritt mit der öffentlichen Bekanntmachung in Kraft.\n\nFreiburg i. Br., den …\n\nMartin W. W. Horn Oberbürgermeister\n\n- 6 -',
#    'f.id': 'https://ris.freiburg.de/oparl/file/vla%7C5160109100198%7C3'}},
#  23: {'questions': ["Welche konkreten Schritte plant die Stadt Freiburg zur Rehabilitierung und Entschädigung von Betroffenen des sogenannten 'Radikalenerlasses'?",
#    'Wie will der Gemeinderat sicherstellen, dass sich junge Menschen heute frei politisch engagieren können, ohne berufliche Nachteile befürchten zu müssen?',
#    'Welche Maßnahmen sind geplant, um das Vertrauen in den öffentlichen Dienst als Arbeitgeber wiederherzustellen und zu stärken?'],
#   'document': {'f.extracted_text': 'Anlage zur DRUCKSACHE G-24/042\n\nEine Stadt für alle – links. ökologisch. feministisch.\nSPD/Kulturliste\nBündnis90/Die Grünen\nJUPI-Fraktion\nFL-Einzelstadtrat\n\nHerrn\nOberbürgermeister Martin Horn\nhpa-ratsbuero@stadt.freiburg.de\n\nim Gemeinderat\n\nFreiburg, 10.10.2023\n\nInterfraktioneller Aufsetzungsantrag nach § 34 GemO – hier: Entschließung des Freiburger Gemeinderats zum sog. „Radikalenerlasses“ bzw. zu den „Berufsverboten“ in Baden-Württemberg\n\nSehr geehrter Herr Oberbürgermeister Horn,\n\nhiermit beantragen die unterzeichnenden Fraktionen und Gemeinderät:innen zur Gemeinderatssitzung am 24.10.2023 folgenden Tagesordnungspunkt aufzunehmen:\n\n„Entschließung des Freiburger Gemeinderats zum sogenannten ‚Radikalenerlasses‘ bzw. zu den ‚Berufsverboten‘ in Baden-Württemberg“\n\nDen Beschlüssen der Gemeinderäte in Heidelberg vom 23.3.2023, Konstanz vom 18.7.2023 und Tübingen vom 24.7.2023 folgend, nehmen wir den 50. Jahrestag des sogenannten Schiess-Erlasses zum Anlass für nachfolgende Entschließung:\n\n„Der Gemeinderat der Stadt Freiburg im Breisgau fordert den Oberbürgermeister auf, sich auf allen Ebenen dafür einzusetzen, dass die Landesregierung Baden-Württembergs den Erlass des Innenministeriums über die Pflicht zur Verfassungstreue im öffentlichen Dienst ("Schiess-Erlass") vom 2. Oktober 1973 ersatzlos und vollumfänglich aufhebt, alle Betroffenen rehabilitiert und entschädigt.“\n\nBegründung:\n\nIm vergangenen Jahr jährte sich zum 50. Mal der sogenannte „Radikalenerlass“. Er wurde 1972 von der Ministerpräsidentenkonferenz unter dem Titel „Grundsätze zur Frage verfassungsfeindlicher Kräfte im Öffentlichen Dienst“ beschlossen. In der Folgezeit wurden etwa 11.000 Berufsverbots- und 2.200 Disziplinarverfahren eingeleitet und offiziell 1.256 Bewerberinnen und Bewerber nicht eingestellt sowie 265 Beamte entlassen.\n\nIn Baden-Württemberg wurde der Beschluss „mit besonderer Härte“ (Ministerpräsident Kretschmann), mittels des sogenannten „Schiess-Erlasses“ vom 2. Oktober 1973 praktiziert. Der nach dem damaligen Innenminister Karl Schiess benannte Erlass jährte sich dieser Tage zum 50. Mal.\n\nDer sogenannte „Radikalenerlass“ hat der Demokratie und dem gesellschaftlichen Klima in der Bundesrepublik schweren Schaden zugefügt. Nicht wenige Menschen wurden in ihrer Existenz bedroht. Sehr viele ließen sich einschüchtern und scheuten ein demokratisches politisches Engagement. Viele der damals Betroffenen spüren die Auswirkungen der Berufsverbote durch Kürzungen bei ihren Ruhegehältern oder sogar Altersarmut bis heute. Einige der Freiburger Betroffenen haben über Jahrzehnte hinweg aktiv an der Gestaltung der kommunalen Demokratie mitgewirkt und zum Beispiel im Freiburger Gemeinderat, den Gewerkschaften oder Bürger:inneninitiativen mitgearbeitet.\n\nNegative Auswirkungen sind auch bis heute in der Gesellschaft spürbar. Nicht wenige junge Menschen, die einen Beruf im öffentlichen Dienst anstreben, sind verunsichert, inwieweit sie sich politisch engagieren dürfen. Leider trifft dies nicht auf Anhänger:innen der rechtsradikalen Szene zu, die sich z.B. im Polizeidienst oder bei der Bundeswehr in den letzten Jahrzehnten bis in Führungspositionen durchaus breit machen konnten.\n\nEine offene, tolerante, demokratische Gesellschaft braucht den uneingeschränkten Erhalt der Grund- und Menschenrechte. Gerade in einer Zeit, in der ein beträchtlicher Teil der Gesellschaft eine äußerst bedenkliche Nähe zu undemokratischen Bewegungen und gruppenbezogener Menschenfeindlichkeit erkennen lässt, ist eine weltoffene, menschenrechtsbezogene Kultur der Vielfalt unverzichtbar.\n\nNach nunmehr 50 Jahren ist es an der Zeit, sich dem Unrecht auch in Baden-Württemberg vollumfänglich zu stellen, die Opfer zu rehabilitieren und zumindest finanziell zu entschädigen, um das Kapitel "Berufsverbote" endgültig abzuschließen.\n\nMit freundlichen Grüßen\n\nIrene Vogel, Stellv. Fraktionsvorsitzende EINE STADT FÜR ALLE\nProf. Günter Rausch, Stadtrat EINE STADT FÜR ALLE\n\nJulia Söhne, Fraktionsvorsitzende SPD/Kulturliste\n\nSimon Sumbert, Co-Fraktionsvorsitzender Bündnis 90/Die Grünen\nPia Federer, Stadträtin Bündnis 90/Die Grünen\n\nDr. Wolf-Dieter Winkler, FL-Einzelstadtrat\n\nSophie Kessl, Stadträtin JUPI-Fraktion\nSergio Pax, Stadtrat JUPI-Fraktion',
#    'f.id': 'https://ris.freiburg.de/oparl/file/vla%7C5420901100041%7C1'}},
#  24: {'questions': ['Wie wird die Rückerstattung von 280.813,47 EUR an die Stadt Freiburg im Haushalt verwendet?',
#    'Welche Auswirkungen hat die reduzierte Zustiftung an die Technologiestiftung BioMed Freiburg auf deren Arbeit und Projekte?',
#    'Wie plant die Stadt, die zurückerhaltenen Mittel für andere Zwecke einzusetzen?'],
#   'document': {'f.extracted_text': 'Anlage 1 zur DRUCKSACHE G-24/003\n\n- Finanzielle Auswirkungen -\n\nBereich: Doppelhaushalt / Stadt Freiburg\nTeilhaushalt / Teilbudget: THH 17 - Stadtkämmerei\nProduktgruppe(n): PG5710\nSeite: 177-179 im Haushaltsplan des DHH 2023/2024\n\nFinanzhaushalt (für investive Maßnahmen) 2023 2024\nEinzahlungen 0 EUR 280.813,47 EUR\nAuszahlungen 0 EUR 0 EUR\nFinanzierungsüberschuss 0 EUR 280.813,47 EUR\n\nEingestellte Mittel im HHPl./MiFi: ☐ in voller Höhe ☐ teilweise ☒ keine\n\nErläuterungen:\nDie Finanzierung der Zustiftung der Stadt Freiburg erfolgt über die bereits im Jahr 2022 der Technologiestiftung BioMed Freiburg treuhänderisch bereitgestellten zweckgebundenen 1.000.000 EUR (siehe Drucksache G-22/219), wobei der nunmehr zu der erforderlichen Zustiftung von 719.186,53 EUR nicht benötigte Differenzbetrag in Höhe von 280.813,47 EUR an die Stadt Freiburg zurückerstattet wird.\n\nDatum der Inbetriebnahme: entfällt',
#    'f.id': 'https://ris.freiburg.de/oparl/file/vla%7C5421810100002%7C1'}}}

In [83]:
for doc_index, content in question_document_mapping.items():
    if doc_index == 2:
        print(f"Document {doc_index}:")
        print("Questions:")
        for question in content['questions']:
            print(f"- {question}")
        print("Document content:")
        print(content['document'])  # Print first 200 characters of the document
        print("\n")
        

Document 2:
Questions:
- Wie haben sich die Übergangsquoten von Grundschulen zu weiterführenden Schulen in Weingarten im Vergleich zum Freiburger Durchschnitt entwickelt?
- Welche konkreten Maßnahmen plant die Stadt, um die Bildungschancen von Schülern aus Weingarten zu verbessern?
- Wie wirkt sich das geplante Bildungshaus ARS auf die Bildungssituation in Weingarten aus?
Document content:
{'f.extracted_text': 'Bildungsangebote in Weingarten\nAusschuss für Schule und Weiterbildung\nKinder- und Jugendhilfeausschuss\nMigrationsausschuss\n13.05.2024\nBildungsangebote in Weingarten/ ASW/ 13.05.2024 Seite 1 12.06.2024\nDezernat II Amt für Schule und Bildung\nIM BREISGAU\n\n1. Ausgangspunkt\nDie Fraktionsanträge im Überblick\nDie Vorbereitung\n2. Die Datenlage\nDas 80%-Narrativ oder was wir tatsächlich zu wissen glauben\n6. Freiburger Bildungsbericht\n3. Erfahrungsberichte\nDer Blick auf Bildungschancen in Weingarten (Wodtke/ Hartwich)\nEine Außenklasse in Weingarten (Wodtke/ Baumgarten)\nBi

In [4]:
from langsmith import Client
import os
import json

# Setup Langsmith client
os.environ["LANGCHAIN_PROJECT"] = "neurabot-ca-evaluation"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
client = Client(api_key="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", api_url="https://eu.api.smith.langchain.com")

# Process the question_document_mapping and create dataset
dataset = []
for doc_index, content in question_document_mapping.items():
    for question in content['questions']:
        dataset.append({
            "input": {"question": question},
            "output": content['document']
        })

# Save to Langsmith
dataset_name = "Freiburg_QA_ID"
langsmith_dataset = client.create_dataset(dataset_name, description="Questions and answers from Pfizer documents")

# Create examples in the dataset
client.create_examples(
    inputs=[item["input"] for item in dataset],
    outputs=[item["output"] for item in dataset],
    dataset_id=langsmith_dataset.id,
)

print(f"Created dataset '{dataset_name}' with {len(dataset)} examples.")

Created dataset 'Freiburg_QA_ID' with 73 examples.
